In [1]:
import pandas as pd
import json
def read_csv_data(file_path):
    """
    Reads the CSV data from the given file path.

    Parameters:
    file_path (str): The path to the CSV file.

    Returns:
    pd.DataFrame: The dataframe containing the CSV data.
    """
    return pd.read_csv(file_path, header=None)
def is_zero(value):
    try:
        return float(value) == 0.0
    except ValueError:
        return False
def find_start_row(data, start_text):
    """
    Finds the start row index where the given text is found.

    Parameters:
    data (pd.DataFrame): The dataframe containing the CSV data.
    start_text (str): The text to search for.

    Returns:
    int: The index of the start row.
    """
    return data[data.apply(lambda row: row.astype(str).str.contains(start_text, case=False, na=False, regex=False).any(), axis=1)].index.min()

def is_header_row(row, header_keywords):
    """
    Checks if a row contains the header keywords.

    Parameters:
    row (pd.Series): The row to check.
    header_keywords (list): The list of header keywords.

    Returns:
    bool: True if the row contains the header keywords, False otherwise.
    """
    return all(keyword in row.tolist() for keyword in header_keywords)

def find_header_indices(data, header_keywords):
    """
    Finds the indices of the rows where the header appears.

    Parameters:
    data (pd.DataFrame): The dataframe containing the CSV data.
    header_keywords (list): The list of header keywords.

    Returns:
    list: The list of indices where the header appears.
    """
    return data[data.apply(is_header_row, axis=1, header_keywords=header_keywords)].index.tolist()

def extract_main_details(table):
    """
    Extracts the main details from the second row of the table.

    Parameters:
    table (pd.DataFrame): The dataframe containing the table data.

    Returns:
    dict: The dictionary containing the main details.
    """
    main_details = {k: v for k, v in table.iloc[0].to_dict().items() if pd.notna(v)}
    last_key = list(main_details.keys())[-1]
    main_details["ANALYSIS NUMBER"] = main_details.pop(last_key)
    return main_details

def extract_sub_table(table, sub_table_start_text):
    """
    Extracts the sub-table that starts with the given text.

    Parameters:
    table (pd.DataFrame): The dataframe containing the table data.
    sub_table_start_text (str): The text indicating the start of the sub-table.

    Returns:
    pd.DataFrame: The dataframe containing the sub-table.
    """
    sub_table_start_row = table[table.apply(lambda row: row.astype(str).str.contains(sub_table_start_text, case=False, na=False, regex=False).any(), axis=1)].index.min()

    if pd.isna(sub_table_start_row):
        return pd.DataFrame()  # Return an empty dataframe if the sub-table start text is not found

    # Extract the sub-table starting from one row below the identified start row (to include the header)
    sub_table = table.iloc[sub_table_start_row:].reset_index(drop=True)

    if sub_table.empty:
        return pd.DataFrame()  # Return an empty dataframe if the sub-table is empty

    # Set the first row as header and remove it from the data
    sub_table.columns = sub_table.iloc[0]
    
    sub_table = sub_table[1:].reset_index(drop=True)

    # Remove columns where all elements are NaN
    sub_table = sub_table.dropna(axis=1, how='all')

    return sub_table

def filter_sub_table(data):
    """
    Filters the sub-table to remove records with NaN in the "NOW CONSUMED" field
    and includes only the specified keys.

    Parameters:
    data (dict): The dictionary containing the 'main_details' and 'sub_table'.

    Returns:
    dict: The filtered dictionary with records removed where "NOW CONSUMED" is NaN
          and includes only the specified keys.
    """
    main_details = data['main_details']
    sub_table = data['sub_table']

    # Create a DataFrame from the sub_table
    sub_table_df = pd.DataFrame(sub_table)

    # Filter out rows where "NOW CONSUMED" is NaN
    sub_table_df = sub_table_df[sub_table_df['NOW CONSUMED'].notna() & ~sub_table_df['NOW CONSUMED'].apply(is_zero)]

    # Specify the required columns
    required_columns = ["NOW CONSUMED", "B/E No", "PER UNIT VALUE"]

    # Ensure the columns are standardized and strip any extra spaces
    sub_table_df.columns = sub_table_df.columns.str.strip()

    # Select only the specified columns
    sub_table_df = sub_table_df[required_columns]

    # Convert the filtered DataFrame back to a list of dictionaries
    filtered_sub_table = sub_table_df.to_dict(orient='records')

    # Return the filtered data
    return {
        'main_details': main_details,
        'sub_table': filtered_sub_table
    }

class CSVDataExtractor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.table1_data = self.extract_table_1()
        self.table492_data = self.extract_table_492()
        self.table957_data = self.extract_table_957()
        
        self.hs_code_wise_tables = self.extract_hs_code_wise_tables()

    def get_analysis_number(self,hs_code):
        for entry in self.table1_data:
            if entry['HS CODE:'] == hs_code:
                return entry['ANALYSIS NO:']
        return None
    def extract_table_1(self):
        # Load the CSV file
        data = pd.read_csv(self.file_path, header=None)
        
        # Find the row where HS CODE starts
        hs_code_row_index = data[data.iloc[:, 0] == 'HS CODE:'].index[0]
        
        # Extract column headers from the row where HS CODE is found
        column_headers = data.iloc[hs_code_row_index, 1:].tolist()
        
        # Rows of interest
        rows_of_interest = ['HS CODE:', 'DESCRIPTION:', 'CTN PCS', 'POLSTER PCS', 'WHITE', 'TOTAL PCS', 'N N WEIGHT:', 'ANALYSIS NO:']
        
        # Extract the rows data
        rows_data = {}
        for row_name in rows_of_interest:
            row_index = data[data.iloc[:, 0] == row_name].index[0]
            rows_data[row_name] = data.iloc[row_index, 1:].tolist()
        
        # Create a list to hold the formatted data
        formatted_data = []
        for col_index in range(len(column_headers)):
            col_data = {row: rows_data[row][col_index] for row in rows_of_interest}
            # Only include entries where 'HS CODE:' is not NaN
            if pd.notna(col_data['HS CODE:']):
                formatted_data.append(col_data)
        
        return formatted_data

    def extract_table_492(self):
        df_full = pd.read_csv(self.file_path)

        # Find the start and end rows for the desired section
        start_text = "CONSUMPTION OF ACCESSORIES IMPORTED UNDER SRO. 492(I)/ 2009"
        end_text = "HS CODE WISE CONSUMPTION"

        # Locate the rows where these headers are found
        start_row = df_full[df_full.apply(lambda row: row.astype(str).str.contains(start_text, case=False, na=False, regex=False).any(), axis=1)].index.max()
        end_row = df_full[df_full.apply(lambda row: row.astype(str).str.contains(end_text, case=False, na=False, regex=False).any(), axis=1)].index.min()

        data_dict = {}  # Initialize an empty dictionary to handle cases where no data is found

        if pd.notna(start_row) and pd.notna(end_row):
            # Read the specific range of rows, skipping the header row itself
            data = pd.read_csv(self.file_path, skiprows=int(start_row)+2, nrows=int(end_row-start_row)-2)
            
            # Filter only the required columns by names
            required_columns = ['B/E No', 'PER UNIT VALUE','Now Consume']
            
            # Check if the required columns are in the data
            if set(required_columns).issubset(data.columns):
                filtered_data = data[required_columns]
                # Filter out rows where 'NOW CONSUMED' is NaN or 0
                filtered_data = filtered_data[filtered_data['Now Consume'].notna() & (filtered_data['B/E No'].notna()) & (filtered_data['Now Consume'] != 0)]
                
                # Convert the DataFrame to a dictionary
                data_dict = filtered_data.to_dict(orient='records')  # 'records' creates a list of dictionaries for each row
        else:
            print("Headers not found in the file")
            raise Exception("Table not found in the file")

        return data_dict

    def extract_table_957(self):
        df_full = pd.read_csv(self.file_path, header=None)

        # Find the start and end rows for the desired section
        start_text = "CONSUMPTION OF RAW MATERIAL  IMPORTED & LOCALY PROCURED  UNDER SRO 957 (I) 2021 DATED. 30-JULY-2021(EFS-EXPORT FACILTAION SCHEME)"
        end_text = "CONSUMPTION OF ACCESSORIES IMPORTED UNDER SRO. 492(I)/ 2009"

        # Locate the rows where these headers are found
        start_row = df_full[df_full.apply(lambda row: row.astype(str).str.contains(start_text, case=False, na=False, regex=False).any(), axis=1)].index.max()
        end_row = df_full[df_full.apply(lambda row: row.astype(str).str.contains(end_text, case=False, na=False, regex=False).any(), axis=1)].index.min()

        data_dict = []  # Initialize an empty list to store the data

        if pd.notna(start_row) and pd.notna(end_row):
            # Read the specific range of rows, skipping the header row itself
            data = pd.read_csv(self.file_path, skiprows=int(start_row)+1, nrows=int(end_row-start_row)-2)
            
            # Extract the headers
            headers = data.columns.tolist()
            
            # Convert headers to strings and strip any leading/trailing spaces
            headers = [str(header).strip() for header in headers]
            
            # Set the new headers
            data.columns = headers
            
            # Extract the relevant columns
            required_columns = ['B/E No/PACKAGE NO/PURCHASE INV#', 'PER UNIT VALUE', 'NOW CONSUMED']
            data = data[required_columns]
            
            # Filter out rows where 'NOW CONSUMED' is NaN or 0
            data = data[data['NOW CONSUMED'].notna() & (data['NOW CONSUMED'] != 0)]
            
            # Convert the DataFrame to a list of dictionaries
            data_dict = data.to_dict(orient='records')
            
            return data_dict
        else:
            print("Headers not found in the file")
            raise Exception("Table 957 not found in the file")
    
    def extract_hs_code_wise_tables(self):
        
        """
        Extracts and formats the tables from the CSV file.

        Parameters:
        file_path (str): The path to the CSV file.

        Returns:
        dict: The dictionary containing the formatted tables.
        """
        # Read the CSV data
        df_full = read_csv_data(self.file_path)

        # Define the header keywords
        header_keywords = ["HS CODE", "DESCRIPTION", "PIECES", "NET NET WEIGHT"]
        start_text = "HS CODE WISE CONSUMPTION"
        sub_table_start_text = "CONSUMPTION OF RAW MATERIAL  IMPORTED UNDER SRO 957 (I) 2021 DATED. 30-JULY-2021(EFS-EXPORT FACILTAION SCHEME)"

        # Find the start row
        start_row = find_start_row(df_full, start_text)
        if pd.isna(start_row):
            raise Exception("Start header 'HS CODE WISE CONSUMPTION' not found in the file")

        # Read the data starting after the "HS CODE WISE CONSUMPTION" row
        data = pd.read_csv(self.file_path, skiprows=int(start_row) + 1, header=None)

        # Find all the rows where the header appears
        header_indices = find_header_indices(data, header_keywords)
        if not header_indices:
            raise Exception("Header row not found in the file")

        tables = {}
        
        for i in range(len(header_indices)):
            start_index = header_indices[i]
            end_index = header_indices[i + 1] if i + 1 < len(header_indices) else len(data)

            # Extract the table
            table = data.iloc[start_index:end_index].reset_index(drop=True)

            # Set the first row as header
            table.columns = table.iloc[0]
            table = table[1:]

            # Remove rows where all elements are NaN
            table = table.dropna(how='all')

            # Extract the main details from the second row
            main_details = extract_main_details(table)
            
            # Extract the sub-table
            sub_table = extract_sub_table(table, sub_table_start_text)
            sub_table_dict = sub_table.to_dict(orient='records') if not sub_table.empty else []

            # Format the final table dictionary
            hs_code = main_details['HS CODE']
            final_table_dict = {
                "main_details": main_details,
                "sub_table": sub_table_dict
            }
            final_table_dict_filtered = filter_sub_table(final_table_dict)
            # Add the formatted table to the tables dictionary
            tables[hs_code] = final_table_dict_filtered

        return tables
path = r"C:\Users\DELL\Downloads\PWSAutomate\EFS-537624.xlsx"
csv = CSVDataExtractor(path)
print(csv)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf0 in position 15: invalid continuation byte

In [1]:
# selenium imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchWindowException, TimeoutException
from selenium.webdriver.support.ui import Select
import time
import math
import argparse
from helpers import *
import os
from datetime import datetime
from selenium.webdriver.chrome.service import Service as ChromeService
from chromedriver_py import binary_path


def setup_driver():
    svc = webdriver.ChromeService(executable_path=binary_path)
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-gpu')
    # chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-software-rasterizer')
    driver = webdriver.Chrome(options=chrome_options, service=svc)

    return driver


def exemptions(driver: webdriver.Chrome):
    # Add SROs/Exemptions
    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgExemptions_ctl02_lnkBtnAdd")
    select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgExemptions_ctl02_ddlExemption",
                    option_text=r"SRO492(I)/2006 -1-0-0-26/05/2006 ( 0 )")
    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgExemptions_ctl02_lnkBtnAdd")

    # Add another
    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgExemptions_ctl03_lnkBtnAdd")
    select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgExemptions_ctl03_ddlExemption",
                    option_text=r"SRO575(I)/02-1-0-0-31/08/2002 ( 0 )")
    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgExemptions_ctl03_lnkBtnAdd")


def assessment_purpose(driver: webdriver.Chrome):
    # Add Assessment Purpose
    text = extract_text(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtQuantity")
    print(f"Extracted Text is {text}")
    write_text(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtQuantity_Statistical_Purpose",
               text=text)
    # write_text(driver=driver,id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtQuantity_International_Traded",text=text)
    # Now Click
    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_btnCalcExportValue")
    time.sleep(3)
    # write_text(driver=driver,id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtActualWeight",text="1002.0000")


def populate_username_passowrd_in_login_form(driver, transaction_id, url, username, password):
    status = False
    error = ''

    try:
        driver.get(url)

        username_input = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[placeholder='Enter Username']"))
        )
        password_input = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[placeholder='Enter Password']"))
        )

        username_input.send_keys(username)
        password_input.send_keys(password)
        sign_in_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Login')]")
        sign_in_button.click()
        status = True
    except TimeoutException:
        error = f"For transaction_id {transaction_id} Login Page did not Appear."
    except Exception as e:
        error = f"For transaction_id {transaction_id} Error Occured => {str(e)}"
    return status, error


def select_types(driver, transaction_id):
    Create_Export_GD = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, "//a[contains(., 'Create Export GD')]"))
    )
    print(f"Found Create_Export_GD.")
    # Perform an action on the input element (e.g., click)
    Create_Export_GD.click()
    select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_GdSelectionExport1_ddlConsignmentType",
                    option_text="Export Facilitation Scheme")
    select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_GdSelectionExport1_ddlDeclarationType",
                    option_text="Export Facilitation Scheme")

    Create_button = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, "(//input[@value='Create'])[2]"))
    )
    print(f"Found Create_button.")
    # Perform an action on the input element (e.g., click)
    Create_button.click()
    print(f"Clicked Create_button.")


def select_saved(driver, transaction_id):
    saved_GD = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_lnkExportSaved"))
    )
    print(f"Found saved_GD.")
    saved_GD.click()
    GD = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_dgSaved_ctl02_lbTrackingId"))
    )
    print(f"Found GD.")
    GD.click()
    GD = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_btnSaveTop"))
    )
    print("GD ready")


def select_GDS(driver, transaction_id):
    status = False
    error = ''
    print(f"For transaction_id {transaction_id} Selecting GDS...")
    try:
        traders_button = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='List of Traders'])[2]"))
        )
        print(f"Found traders_button.")
        traders_button.click()
        iframe = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
        )
        driver.switch_to.frame(iframe)
        print(f"iframe Switched.")
        time.sleep(3)
        company_element = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located(
                (By.XPATH, "//td[text()='STYLE TEXTILE (PRIVATE) LTD']/preceding-sibling::td[1]//input[@type='image']"))
        )
        print(f"Found company_element.")
        # Perform an action on the input element (e.g., click)
        company_element.click()
        driver.switch_to.default_content()
        time.sleep(3)
        print(f"Switched to default_content.")
        goods_declaration = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='Goods Declaration'])[2]"))
        )
        print(f"Found goods_declaration.")
        goods_declaration.click()
        driver.switch_to.frame(iframe)
        select_types(driver, transaction_id)
        # select_saved(driver,transaction_id)
        status = True
    except Exception as e:
        error = f"For transaction_id {transaction_id} Error Occured => {str(e)}"
        print(e)
    return status, error


def fill_form(driver: webdriver.Chrome, transaction_id, data={}):
    status = False
    error = ''
    pl_data = data.get("pl_data")
    fty_data = data.get("fty_data")
    print(f"For transaction_id {transaction_id} Filling the Form...")
    try:
        # Basic Info
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_BasicInfoUc1_ddlCollectorate",
                        option_text="Port Qasim (exports), karachi")
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_BasicInfoUc1_ddlModeOfTransport",
                        option_text="Inland water Transport")

        # Consignor & Consignee Information (Hide)
        write_text(driver, "ctl00_ContentPlaceHolder2_ConsigneeInfoUc1_txtConsignorName", pl_data.get("consignee"))
        write_text(driver, "ctl00_ContentPlaceHolder2_ConsigneeInfoUc1_txtConsignorAddress", pl_data.get("address"))

        # GD Information
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_GdInfoSeaUc_ddlConsignmentMode",
                        option_text="Part Shipment")
        # SET the BL/AWB No (Invoice No Last Six digits)
        invoices = data.get("final_table").get('invoices')
        bl_awb_no = invoices.split("\n")[0]
        write_text(driver, "ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtBlNo",
                   bl_awb_no)
        #  SET the BL/AWB Date (Invoice Date)
        # Parse the date string into a datetime object
        date_object = datetime.strptime(data.get("final_table").get('date'), "%Y-%m-%d")
        write_date(driver, "ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtBlDate_txtDate", date_object.strftime("%d/%m/%Y"))
        #  Select the Port of Shipment dropdown
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_GdInfoSeaUc_ddlPortOfShipment",
                        option_text="Port Qasim (exports), karachi")
        #  Select the Destination Country
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_GdInfoSeaUc_ddlDestinationCountry",
                        option_text="United States")
        #  Select the Destination Port
        select_dropdown_by_value(driver=driver, id="ctl00_ContentPlaceHolder2_GdInfoSeaUc_ddlPortOfDischarge",
                                 value="41846")
        # Select the Shipping Line
        select_dropdown_by_value(driver=driver, id="ctl00_ContentPlaceHolder2_GdInfoSeaUc_ddlShippingline", value="66")
        # SET Place of Delivery
        write_text(driver, "ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtPlaceofDelivery", "Long Beach, USA")
        # SET Net Weight (MT)
        net_weight = data.get("final_table").get("Net Weight") / 1000
        write_text(driver, "ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtNetWeight", net_weight)
        # SET Gross Weight
        gross_weight = data.get("final_table").get("Gross Weight") / 1000
        write_text(driver, "ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtGrossWeight", gross_weight)
        # SET Marks
        marks = f"""AS PER SHIPPER \n INVOICE NO.\n{invoices}"""
        write_text(driver, "ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtMarks", marks)

        # financials_info
        # Select the PAYMENT TERM dropdown
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_FinancialsUc1_ddlPaymentTerms",
                        option_text="Without LC")
        # Select the Bank dropdown
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_FinancialsUc1_ddlBank",
                        option_text="MCB BANK LIMITED")
        # Select the Delivery Term dropdown
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_FinancialsUc1_ddlDeliveryTerm",
                        option_text="Free On Board (FOB)")
        # Select the Currency dropdown
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_FinancialsUc1_ddlCurrency",
                        option_text="United States-US $")
        # SET FOB Value
        write_text(driver, "ctl00_ContentPlaceHolder2_FinancialsUc1_txtFobValue",
                   data.get("final_table").get("PO Net Amount"))

        # supporting_info_fill

        # Select the Shed/Location Code dropdown
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_SupportingInfoUc1_ddlExaminerGroup",
                        option_text="IC3")
        # Select the ddlTerminal dropdown
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_SupportingInfoUc1_ddlTerminal",
                        option_text="Qasim International Container Terminal")

        # check_disclaimer
        checkbox(driver, id="ctl00_ContentPlaceHolder2_ExportPolicyDisclaimerUc1_chkExportPolicy")
        checkbox(driver, id="ctl00_ContentPlaceHolder2_ExportPolicyDisclaimerUc1_chkIsSroDisclaimer")

        # Click the Save button
        click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnSaveBottom")

        print(f"GD Filling Success")
    except Exception as e:
        error = f"For transaction_id {transaction_id} Error Occured => {str(e)}"
        print(e)
    return status, error


def upload_document(driver: webdriver.Chrome, filepath: os.path.join):
    UploadButton = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH,
                                        "//a[@id='ctl00_ContentPlaceHolder2_UploadDocumentUc1_lnkUploadDocument' and text()='Upload Document']"))
    )
    print(f"Found Upload Button.")
    UploadButton.click()
    fileButton = WebDriverWait(driver, 1000).until(
        EC.presence_of_element_located((By.XPATH,
                                        "//table[.//span[text()='Attached Document']]//input[@id='ctl00_ContentPlaceHolder2_GdExportUploadDocUc1_GdImportDocUpload_fuDoc']"))
    )
    print("fileButton ready")
    fileButton.send_keys(filepath)
    if 'fty' in str(filepath).lower():
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_GdExportUploadDocUc1_cmbDocumentType",
                        option_text="Invoice")
    else:
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_GdExportUploadDocUc1_cmbDocumentType",
                        option_text="Packing List")
    UploadDocButton = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH,
                                        "//table[.//span[text()='Attached Document']]//input[@id='ctl00_ContentPlaceHolder2_GdExportUploadDocUc1_btnUpload' and @value='Upload']"))
    )
    print(f"Found Upload DOC Button.")
    UploadDocButton.click()

    WebDriverWait(driver, 500).until(
        EC.presence_of_element_located((By.XPATH,
                                        "//table[.//span[text()='Customs Office']]//select[@id='ctl00_ContentPlaceHolder2_BasicInfoUc1_ddlCollectorate']"))
    )
    print("GD ready")


def upload_documents(driver: webdriver.Chrome, pdf_path):
    for file in pdf_path:
        print(f"Uploading file: {file}")
        upload_document(driver, file)


def fill_container_info(driver: webdriver.Chrome, data={}):
    # Now Click on Container Info

    click_button(driver=driver,
                 id="//table[.//span[text()='Containers Information']]//a[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgContainer_ctl02_lnkBtnAdd']",
                 by=By.XPATH)

    write_text(driver=driver,
               id="//table[.//span[text()='Containers Information']]//input[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgContainer_ctl02_txtContainerNo']",
               text="STYLE123456", by=By.XPATH)
    quantity = str(data.get('Quantity'))
    write_text(driver=driver,
               id="//table[.//span[text()='Containers Information']]//input[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgContainer_ctl02_txtQuantity']",
               text=quantity, by=By.XPATH)
    cartons = str(data.get('Carton'))
    write_text(driver=driver,
               id="//table[.//span[text()='Containers Information']]//input[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgContainer_ctl02_txtNoOfPackages']",
               text=cartons, by=By.XPATH)
    select_dropdown(driver=driver,
                    id="//table[.//span[text()='Containers Information']]//select[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgContainer_ctl02_ddlPackageType']",
                    option_text="CARTONS", by=By.XPATH)
    click_button(driver=driver,
                 id="//table[.//span[text()='Containers Information']]//a[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgContainer_ctl02_lnkBtnAdd']",
                 by=By.XPATH)


def add_item(driver: webdriver.Chrome, transaction_id, data={},item_no=None):
    print(f"For transaction_id {transaction_id} Adding Item...")
    click_button(driver=driver, id="//a[@id='ctl00_ContentPlaceHolder2_ItemInfoUc1_lnkItems' and text()='Add Items']",
                 by=By.XPATH)
    print("Clicked Add Items")
    write_text(driver, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtHsCode", data.get('hs_code'))
    description = data.get('description')
    if item_no:
        additional_text = "UNDER CLAIM FOR 'DRAWBACK' OF LOCAL TAXES AND BRAND NOTIFICATION NO. 3(1) TID/09-P-I DATED: 01.09.2009 IMPORTED MATERIAL USED UNDER EFS SRO 957(I)21 DT.30.07.2021) (DETAILS AS PER INVOICE) INVOICE NO."
        description = f"{description} \n {additional_text} {data.get('invoice_number')}"
    else:
        additional_text = "(DETAILS AS PER INVOICE) INVOICE NO. "
        description = f"{description} \n {additional_text} {data.get('invoice_number')}"
    write_text(driver, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtDeclaredDescription", description)
    select_dropdown_by_value(driver, 'ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_ddlOrigion', '586')
    unit_value = (data.get('PO Net Amount')) / (data.get('Quantity'))
    write_text(driver, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtUnitValue", unit_value)
    write_text(driver, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtActualWeight", data.get('Quantity'))
    # Select Actual Unit now
    select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_ddlActualWeightUnit",
                    option_text="NO")
    print("filling the exemptions")
    exemptions(driver)
    # Check	BLEACHED BLENDED GARMENTS, WEARING APPAREL (ALL BLENDS OF POLYESTER STAPLE FIBRE AND COTTON FIBRE).
    process_duty_drawback(driver, data.get('description',''))
    # Now Handel Quantity (for Assessment Purpose)
    time.sleep(5)
    WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH,
                                        "//table[.//span[text()='HS Code']]//input[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtHsCode']"))
    )
    print(f"Now Executing assessment_purpose")

    fill_container_info(driver=driver, data=data)
    assessment_purpose(driver)
    # Now Click on Save Button
    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnSaveBottom")
    WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_BasicInfoUc1_pnlTitle"))
    )
    print("HScode Added")
    return data.get('hs_code')


def process_gd_number_pop_up_492(driver : webdriver.Chrome,data):
            # Store the ID of the original window
    original_window = driver.current_window_handle

    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_btnGDLookup")

    # Wait for the new window or tab (assume we know a new window opens here)
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Get the list of all window handles
    windows = driver.window_handles

    # Switch to the new window
    for window in windows:
        if window != original_window:
            driver.switch_to.window(window)
            break

    # Now you can interact with the new window
    # For example, finding an element and interacting with it
    write_text(driver, "txtSearch",data.get('B/E No'),pop_up=True)
    click_button(driver, "btnSearch",pop_up=True)
    time.sleep(5)
    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "tblAlert"))
    )
    if 'no data found' in table.text.lower():
        try:
            driver.close()
            driver.switch_to.window(original_window)
            iframe = WebDriverWait(driver, 100).until(
                    EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
                )
            driver.switch_to.frame(iframe)
        except NoSuchWindowException:
            print("The new window was already closed.")
        return None
    click_button(driver, id="//tr[@class='ItemStyle']//a[@id='dgLookup_ctl02_lbSelect']",by=By.XPATH,pop_up=True)
        # Attempt to close the new window
    try:
        driver.close()
    except NoSuchWindowException:
        print("The new window was already closed.")

    # Switch back to the original window
    driver.switch_to.window(original_window)
    iframe = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
        )
    driver.switch_to.frame(iframe)
    # Locate the table
    table = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_dgItems"))
        )

    # Find all rows in the table body
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Iterate through the rows, skipping the header
    for row in rows[1:]:
        cells = row.find_elements(By.TAG_NAME, "td")
        if cells:
            try:
                unit_value = float(cells[4].text)
                puv = float(data.get('PER UNIT VALUE'))
                if math.isclose(unit_value, puv, rel_tol=0.1):
                    select_link = cells[0].find_element(By.TAG_NAME, "a")
                    if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
                        select_link.click()
                    else:
                        return None
                    time.sleep(2)
                    break
            except ValueError:
                print(f"Skipping row due to non-numeric value: {cells[4].text}")
    else:
        # select 1st row
        row = rows[1]
        cells = row.find_elements(By.TAG_NAME, "td")
        if cells:
            select_link = cells[0].find_element(By.TAG_NAME, "a")
            print(f"No matching row found in the table for PER UNIT VALUE {data.get('PER UNIT VALUE')} and {data.get('B/E No')} Selecting 1st row")
            if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
                select_link.click()
            else:
                print("Select Link is not enabled")
                return None
        else:
            return None
    try:
        Quantity = float(extract_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtQuantity"))
    except :
        Quantity = 0.0
    try:
        CONSUMED = float(data.get('Now Consume'))
    except :
        CONSUMED = 0.0
    if CONSUMED < Quantity:
        write_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtQuantity",data.get('Now Consume'),pop_up=True)
    return True

def process_gd_number_pop_up_957(driver : webdriver.Chrome,data,is_hscode_wise=False):
    be_no = 'B/E No/PACKAGE NO/PURCHASE INV#'
    if is_hscode_wise:
        be_no = 'B/E No'
            # Store the ID of the original window
    original_window = driver.current_window_handle

    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_btnGDLookup")

    # Wait for the new window or tab (assume we know a new window opens here)
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Get the list of all window handles
    windows = driver.window_handles

    # Switch to the new window
    for window in windows:
        if window != original_window:
            driver.switch_to.window(window)
            break

    # Now you can interact with the new window
    # For example, finding an element and interacting with it
    write_text(driver, "txtSearch",data.get(be_no),pop_up=True)
    click_button(driver, "btnSearch",pop_up=True)
    time.sleep(5)
    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "tblAlert"))
    )
    if 'no data found' in table.text.lower():
        try:
            driver.close()
            driver.switch_to.window(original_window)
            iframe = WebDriverWait(driver, 100).until(
                    EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
                )
            driver.switch_to.frame(iframe)
        except NoSuchWindowException:
            print("The new window was already closed.")
        return None
    click_button(driver, id="//tr[@class='ItemStyle']//a[@id='dgLookup_ctl02_lbSelect']",by=By.XPATH,pop_up=True)
        # Attempt to close the new window
    try:
        driver.close()
    except NoSuchWindowException:
        print("The new window was already closed.")

    # Switch back to the original window
    driver.switch_to.window(original_window)
    iframe = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
        )
    driver.switch_to.frame(iframe)
    # Locate the table
    table = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_dgItems"))
        )

    # Find all rows in the table body
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Iterate through the rows, skipping the header
    for row in rows[1:]:
        cells = row.find_elements(By.TAG_NAME, "td")
        if cells:
            print("The celss text is : ",cells[4].text)
            try:
                unit_value = float(cells[4].text)
                puv = float(data.get('PER UNIT VALUE'))
                if math.isclose(unit_value, puv, rel_tol=0.1):
                    select_link = cells[0].find_element(By.TAG_NAME, "a")
                    if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
                        select_link.click()
                    else:
                        return None
                    time.sleep(2)
                    break
            except ValueError:
                print(f"Skipping row due to non-numeric value: {cells[4].text}")
    else:
        print('In rows')
        # select 1st row
        row = rows[1]
        cells = row.find_elements(By.TAG_NAME, "td")
        if cells:
            select_link = cells[0].find_element(By.TAG_NAME, "a")
            print(f"No matching row found in the table for PER UNIT VALUE {data.get('PER UNIT VALUE')} and {data.get(be_no)} Selecting 1st row")
            if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
                select_link.click()
                time.sleep(5)
            else:
                print("Select Link is not enabled")
                return None
        else:
            return None
    try:
        Quantity = float(extract_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtQuantity"))
    except :
        Quantity = 0.0
    try:
        CONSUMED = float(data.get('NOW CONSUMED'))
    except :
        CONSUMED = 0.0
    if CONSUMED < Quantity:
        write_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtQuantity",data.get('NOW CONSUMED'),pop_up=True)
    hs_code = extract_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtHsCode")
    return hs_code

def process_analysis_number_pop_up_957(driver : webdriver.Chrome,analysis_number,hs_code):
            # Store the ID of the original window
    original_window = driver.current_window_handle

    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_btnAnalysisLookup")

    # Wait for the new window or tab (assume we know a new window opens here)
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Get the list of all window handles
    windows = driver.window_handles

    # Switch to the new window
    for window in windows:
        if window != original_window:
            driver.switch_to.window(window)
            break

    # Now you can interact with the new window
    # For example, finding an element and interacting with it
    write_text(driver, "txtSearch",analysis_number,pop_up=True)
    write_text(driver, "txtInputHSCode",hs_code,pop_up=True)
    click_button(driver, "btnSearch",pop_up=True)
    time.sleep(3)
    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "tblAlert"))
    )
    if 'no data found' in table.text.lower():
        try:
            driver.close()
            driver.switch_to.window(original_window)
            iframe = WebDriverWait(driver, 100).until(
                    EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
                )
            driver.switch_to.frame(iframe)
        except NoSuchWindowException:
            print("The new window was already closed.")
        return None
    click_button(driver, id="//tr[@class='ItemStyle']//a[@id='dgLookupExport_ctl02_lbSelect']",by=By.XPATH,pop_up=True)
        # Attempt to close the new window
    try:
        driver.close()
    except NoSuchWindowException:
        print("The new window was already closed.")

    # Switch back to the original window
    driver.switch_to.window(original_window)
    iframe = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
        )
    driver.switch_to.frame(iframe)
    time.sleep(2)
    wait_for_page_load(driver)
    return True

def add_excel_data_492(driver: webdriver.Chrome, data):
    # Wait until the image is present
    try:
        toggle_NonDutyPaid(driver)
        click_button(driver=driver,
                     id="//a[@id='ctl00_ContentPlaceHolder2_NonDutyPaidItemInfoUc1_lnkItems' and text()='Attach Item']",
                     by=By.XPATH)
        pop_up_492 = process_gd_number_pop_up_492(driver, data)
        if pop_up_492:
            click_button(driver=driver, id="//input[@id='ctl00_ContentPlaceHolder2_btnSaveBottom']", by=By.XPATH)
            table = WebDriverWait(driver, 100).until(
                        EC.presence_of_element_located((By.ID, "tblAlert"))
                    )
            error_text = table.text.lower().strip()
            if error_text:
                raise Exception(f"Got Error for {data.get('B/E No')}")
            WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_pnlTitle"))
            )
            print(f"Element in pop up Added.")
        else:
            print(f"HS Code Not Found for {data.get('B/E No')}")
            # Cancel the present filling
            click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnCancelBottom")
    except:
        print(f"Got Error for {data.get('B/E No')}")
        # Cancel the present filling
        click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnCancelBottom")
        WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_pnlTitle"))
        )

def add_excel_data_957(driver: webdriver.Chrome, data, analysis_number, is_hscode_wise=False):
    be_no = 'B/E No/PACKAGE NO/PURCHASE INV#'
    if is_hscode_wise:
        be_no = 'B/E No'
    toggle_NonDutyPaid(driver)
    click_button(driver=driver,
                 id="//a[@id='ctl00_ContentPlaceHolder2_NonDutyPaidItemInfoUc1_lnkItems' and text()='Attach Item']",
                 by=By.XPATH)
    try:
        hs_code = process_gd_number_pop_up_957(driver, data, is_hscode_wise)
        if hs_code:
            res_process_analysis_number_pop_up_957 = process_analysis_number_pop_up_957(driver, analysis_number=analysis_number, hs_code=hs_code)
            if res_process_analysis_number_pop_up_957:
                click_button(driver=driver, id="//input[@id='ctl00_ContentPlaceHolder2_btnSaveBottom']", by=By.XPATH)
                table = WebDriverWait(driver, 100).until(
                        EC.presence_of_element_located((By.ID, "tblAlert"))
                    )
                error_text = table.text.lower().strip()
                if error_text:
                    print(f"Got Error for {data.get(be_no)}")
                    raise Exception(f"Got Error for {data.get(be_no)}")
                print(f"Element in pop up Processed.")
            else:
                raise Exception(f"Analysis Number Not Found for {data.get(be_no)}")
                
        else:
            raise Exception(f"HS Code Not Found for {data.get(be_no)}")
    except Exception as e:
        print(f"Got Error for {data.get(be_no)} => {str(e)}")
        # Cancel the present filling
        click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnCancelBottom")
        WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_pnlTitle"))
        )


def process_492(driver, data):
    for idx, obj in enumerate(data):
        if obj.get('B/E No'):
            add_excel_data_492(driver, data=obj)


def process_957(driver, data, analysis_number, is_hscode_wise=False):
    be_no = 'B/E No/PACKAGE NO/PURCHASE INV#'
    if is_hscode_wise:
        be_no = 'B/E No'
    for obj in data:
        if categorize_invoice(obj.get(be_no)) == 'non_local':
            add_excel_data_957(driver, data=obj, analysis_number=analysis_number, is_hscode_wise=is_hscode_wise)
        else:
            print(f"Local Invoice {obj.get(be_no)}")
            add_excel_data_local(driver, data=obj, analysis_number=analysis_number, is_hscode_wise=is_hscode_wise)


def process_localy_purchased_analysis_no_pop_up_957(driver: webdriver.Chrome, analysis_number, hs_code):
    # Store the ID of the original window
    original_window = driver.current_window_handle

    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_LocalPurchaseItemEntryUc1_btnAnalysisLookup")

    # Wait for the new window or tab (assume we know a new window opens here)
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Get the list of all window handles
    windows = driver.window_handles

    # Switch to the new window
    for window in windows:
        if window != original_window:
            driver.switch_to.window(window)
            break

    # Now you can interact with the new window
    # For example, finding an element and interacting with it
    write_text(driver, "txtSearch", analysis_number, pop_up=True)
    write_text(driver, "txtInputHSCode", hs_code, pop_up=True)
    click_button(driver, "btnSearch", pop_up=True)
    time.sleep(3)
    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "tblAlert"))
    )
    if 'no data found' in table.text.lower():
        try:
            driver.close()
            driver.switch_to.window(original_window)
            iframe = WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
            )
            driver.switch_to.frame(iframe)
        except NoSuchWindowException:
            print("The new window was already closed.")
        return None
    click_button(driver, id="//tr[@class='ItemStyle']//a[@id='dgLookupExport_ctl02_lbSelect']", by=By.XPATH,
                 pop_up=True)
    # Attempt to close the new window
    try:
        driver.close()
    except NoSuchWindowException:
        print("The new window was already closed.")

    # Switch back to the original window
    driver.switch_to.window(original_window)
    iframe = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
    )
    driver.switch_to.frame(iframe)
    wait_for_page_load(driver)
    return True


def process_localy_purchased_pop_up_957(driver: webdriver.Chrome, data, is_hscode_wise=False):
    be_no = 'B/E No/PACKAGE NO/PURCHASE INV#'
    if is_hscode_wise:
        be_no = 'B/E No'
        # Store the ID of the original window
    original_window = driver.current_window_handle

    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_LocalPurchaseItemEntryUc1_btnLocalPurchaseLookup")

    # Wait for the new window or tab (assume we know a new window opens here)
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Get the list of all window handles
    windows = driver.window_handles

    # Switch to the new window
    for window in windows:
        if window != original_window:
            driver.switch_to.window(window)
            break

    # Now you can interact with the new window
    # For example, finding an element and interacting with it
    write_text(driver, "txtSearch", data.get(be_no), pop_up=True)
    click_button(driver, "btnSearch", pop_up=True)
    time.sleep(5)
    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "tblAlert"))
    )
    if 'no data found' in table.text.lower():
        try:
            driver.close()
            driver.switch_to.window(original_window)
            iframe = WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
            )
            driver.switch_to.frame(iframe)
        except NoSuchWindowException:
            print("The new window was already closed.")
        return None
    click_button(driver, id="//tr[@class='ItemStyle']//a[@id='dgLookup_ctl02_lbSelect']", by=By.XPATH, pop_up=True)
    # Attempt to close the new window
    try:
        driver.close()
    except NoSuchWindowException:
        print("The new window was already closed.")

    # Switch back to the original window
    driver.switch_to.window(original_window)
    iframe = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
    )
    driver.switch_to.frame(iframe)
    # Locate the table

    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_LocalPurchaseItemEntryUc1_dgItems"))
    )

    # Find all rows in the table body
    rows = table.find_elements(By.TAG_NAME, "tr")

    row = rows[1]
    cells = row.find_elements(By.TAG_NAME, "td")
    if cells:

        select_link = cells[0].find_element(By.TAG_NAME, "a")
        print(
            f"No matching row found in the table for PER UNIT VALUE {data.get('PER UNIT VALUE')} and {data.get(be_no)} Selecting 1st row")
        if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
            select_link.click()
            time.sleep(2)
        else:
            print("Select Link is not enabled")
            return None
    else:
        return None
    hs_code = extract_text(driver, "ctl00_ContentPlaceHolder2_LocalPurchaseItemEntryUc1_txtHsCode")
    return hs_code


def add_excel_data_local(driver: webdriver.Chrome, data, analysis_number, is_hscode_wise=False):
    be_no = 'B/E No/PACKAGE NO/PURCHASE INV#'
    if is_hscode_wise:
        be_no = 'B/E No'
    # Wait until the image is present
    toggle_LocalPurchaseItem(driver)
    click_button(driver=driver,
                 id="//a[@id='ctl00_ContentPlaceHolder2_LocalPurchaseItemEntryInfoUc1_lnkItems' and text()='Attach Locally Purchase Item']",
                 by=By.XPATH)
    time.sleep(2)

    hs_code = process_localy_purchased_pop_up_957(driver, data, is_hscode_wise)
    if hs_code:
        res_957 = process_localy_purchased_analysis_no_pop_up_957(driver, analysis_number=analysis_number,
                                                                  hs_code=hs_code)
        if res_957:
            click_button(driver=driver, id="//input[@id='ctl00_ContentPlaceHolder2_btnSaveBottom']", by=By.XPATH)
            time.sleep(1)
        else:
            # Cancel the present filling
            click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnCancelBottom")
            time.sleep(1)
        WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_LocalPurchaseItemEntryInfoUc1_lblTitle"))
        )
        print(f"Element in pop up Added.")
    else:
        print(f"HS Code Not Found for {data.get(be_no)}")
        # Cancel the present filling
        click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnCancelBottom")


def Non_Duty_Paid_Info(driver, csv_obj: CSVDataExtractor, hs_code, elem_index):
    select_added_item(driver)
    # Wait until the image is present

    data_492 = csv_obj.table492_data
    data_957 = csv_obj.table957_data

    process_492(driver, data_492)
    analysis_number = csv_obj.get_analysis_number(hs_code)
    process_957(driver, data_957, analysis_number)
    time.sleep(5)
    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnSaveBottom")
    WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_BasicInfoUc1_pnlTitle"))
    )


def Non_Duty_Paid_Info_multi_po(driver, csv_obj: CSVDataExtractor, hs_code, elem_index):
    select_added_item(driver)
    toggle_NonDutyPaid(driver)

    data_957_hs_code_wise = csv_obj.hs_code_wise_tables.get(hs_code)
    data_957 = data_957_hs_code_wise.get('sub_table')

    analysis_number = csv_obj.get_analysis_number(hs_code)
    process_957(driver, data_957, analysis_number, is_hscode_wise=True)
    time.sleep(5)
    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnSaveBottom")
    WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_BasicInfoUc1_pnlTitle"))
    )


def process_multi_single(driver, items_data, prev_idx=0):
    for idx, item_data in enumerate(items_data):
        item_no=True if idx == 0 else None
        hs_code = add_item(driver, 1, data=item_data, item_no=item_no)
        Non_Duty_Paid_Info(driver, item_data.get('csv_obj'), hs_code, elem_index=idx + 1 + prev_idx)
        print(f"GD Completed For Item : {item_data}")
    return idx


def process_multi_po(driver, po_obj: MultiPOParse):
    po_tables = po_obj.extracted_data.get('po_tables')
    invoice_nos = po_tables.keys()
    print(invoice_nos)

    for invoice in invoice_nos:
        processed_hs_codes = set()
        items_data = po_tables[invoice].get('po_numbers')
        print(items_data)
        for idx, item_data in enumerate(items_data):
            
            data_to_send = po_obj.get_item_info(int(item_data))
            totals = po_tables[invoice].get('totals')
            csv_obj = po_tables[invoice].get('csv_obj')
            data_to_send['invoice_number'] = str(invoice)
            data_to_send['csv_obj'] = csv_obj
            data_to_send.update(totals)
            data_to_send['Carton'] = data_to_send['CTNS']
            data_to_send['Quantity'] = data_to_send['PCS']
            data_to_send['PO Net Amount'] = data_to_send['PO_Amount']
            print(data_to_send)

            hs_code = data_to_send.get('hs_code')

            if not hs_code:
                continue

            if idx == 0:
                add_item(driver, 1, data=data_to_send, item_no=True)
            else:
                add_item(driver, 1, data=data_to_send)

            if hs_code not in processed_hs_codes:
                if idx == 0:
                    Non_Duty_Paid_Info(driver, data_to_send.get('csv_obj'), hs_code=hs_code, elem_index=idx + 1)
                    print(f"Adding HS Code Wise Tables for hs code {hs_code}")
                Non_Duty_Paid_Info_multi_po(driver, data_to_send.get('csv_obj'), hs_code=hs_code, elem_index=idx + 1)
                print(f"GD Completed For Item: {item_data}")
                processed_hs_codes.add(hs_code)
            else:
                print(f"HS code {hs_code} is duplicate, skipping Non_Duty_Paid_Info.")

    return idx

def main(data):
    try:
        print('Starting the process')
        finalStatus = False
        finalMessage =''
        driver = setup_driver()
        transaction_id = data.get("transaction_id")
        username = data["UserName"]
        passwd = data["Password"]
        url = data["URL"]
        login_form_status = True
        login_form_status, login_form_error = populate_username_passowrd_in_login_form(driver, transaction_id,url, username, passwd)
        if login_form_status:
            # pickle.dump(driver.get_cookies(), open("cookies.pkl", "wb"))
            print(f"For transaction_id { transaction_id } Cookies Done...")
            wait = WebDriverWait(driver, 60)
            # Define a condition to wait for page load (you can customize this)
            page_load_condition = EC.presence_of_element_located((By.ID, "sidebar-container"))
            # Wait until the page is fully loaded
            wait.until(page_load_condition)
            time.sleep(3)
            GD_status, GD_error = select_GDS(driver, transaction_id)
            print(f"GD_status: {GD_status}, GD_error: {GD_error}")
            if GD_status:
                fill_form_status, fill_form_error = fill_form(driver, transaction_id,data=data)
                time.sleep(5)
                print(f"Uploading the documents")
                upload_documents(driver,pdf_path=data.get('pdf_paths'))
                time.sleep(5)
                fty_data = data.get('fty_data')
                po_obj = data.get('po_obj')
                
                prev_idx = 0
                print(f"Starting the Multi PO Process")
                if po_obj:
                    print(f"Starting the Multi PO Process")
                    prev_idx = process_multi_po(driver,po_obj)
                if fty_data:
                    items_data = fty_data.get('extracted_data')
                    print(f"Now Starting the Multi Single Process")
                    process_multi_single(driver,items_data,prev_idx)
        else:
            finalMessage = login_form_error

    except Exception as e:
        print(f"For transaction_id { transaction_id } Exception occurred: {e}")
        finalStatus = False
        finalMessage = str(e)
    finally:
        return driver
pl_pdf_path, fty_pdf_path, csv_path = extract_files_club_single()
pl_po_pdf_path, fty_po_pdf_path, csv_po_path,desc_po_path = extract_files_club_po()

fty_parser = MultiSingleParse(fty_pdf_path,csv_path=csv_path)
pl_parser = PlParse(pl_po_pdf_path)
po_parser = MultiPOParse(path=fty_po_pdf_path,csv_path=csv_po_path,des_path=desc_po_path)
pdf_paths = [pl_pdf_path, fty_pdf_path]
final_table_data = add_data_dictionaries(po_parser.extracted_data['final_table'],fty_parser.extracted_data['final_table'])

data = {
    'transaction_id': 1,
    'user_id': 2,
    'URL': 'https://app.psw.gov.pk/app/',
    'UserName': 'CA-01-2688539',
    'Password': 'Express@5599',
    'pdf_paths': pdf_paths,
    'pl_data':pl_parser.extracted_data,
    'fty_data':fty_parser.extracted_data,
    'po_obj':po_parser,
    'final_table':final_table_data,
}
print(fty_parser.extracted_data['extracted_data'][0].get('csv_obj').table492_data)
print(fty_parser.extracted_data['extracted_data'][0].get('csv_obj').table957_data)
print(fty_parser.extracted_data['extracted_data'][0].get('csv_obj').table1_data)
required_keys = list(data.keys())

driver = main(data)


PL PDF Path: c:\Users\DELL\Downloads\PWSAutomate\multi\multisingle\pl.pdf
FTY PDF Path: c:\Users\DELL\Downloads\PWSAutomate\multi\multisingle\fty.pdf
CSV Path: c:\Users\DELL\Downloads\PWSAutomate\multi\multisingle
PL PDF Path: c:\Users\DELL\Downloads\PWSAutomate\multi\multipos\pl.pdf
FTY PDF Path: c:\Users\DELL\Downloads\PWSAutomate\multi\multipos\fty3.pdf
CSV Path: c:\Users\DELL\Downloads\PWSAutomate\multi\multipos
adidas branded garments. export from export facilitation scheme (efs) 134740069 unisex infants 78% cotton 22% recycled polyesterknit pullover dyed
Women 80/20% Pullovers
Searching for CSV file for invoice number 507944 in c:\Users\DELL\Downloads\PWSAutomate\multi\multisingle...
EFS507944.csv
adidas branded garments. export from export facilitation scheme (efs) 134743455 womens 70% cotton 30% recycle polyster knit t-shirts dyed
Women 80/20% T-Shirts
Searching for CSV file for invoice number 507957 in c:\Users\DELL\Downloads\PWSAutomate\multi\multisingle...
EFS507944.csv
EFS5

In [1]:
# selenium imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchWindowException, TimeoutException
from selenium.webdriver.support.ui import Select
import time
import math
import argparse
from helpers import *
import os
from datetime import datetime
from selenium.webdriver.chrome.service import Service as ChromeService
from chromedriver_py import binary_path


def setup_driver():
    svc = webdriver.ChromeService(executable_path=binary_path)
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-gpu')
    # chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-software-rasterizer')
    driver = webdriver.Chrome(options=chrome_options, service=svc)

    return driver


def exemptions(driver: webdriver.Chrome):
    # Add SROs/Exemptions
    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgExemptions_ctl02_lnkBtnAdd")
    select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgExemptions_ctl02_ddlExemption",
                    option_text=r"SRO492(I)/2006 -1-0-0-26/05/2006 ( 0 )")
    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgExemptions_ctl02_lnkBtnAdd")

    # Add another
    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgExemptions_ctl03_lnkBtnAdd")
    select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgExemptions_ctl03_ddlExemption",
                    option_text=r"SRO575(I)/02-1-0-0-31/08/2002 ( 0 )")
    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgExemptions_ctl03_lnkBtnAdd")


def assessment_purpose(driver: webdriver.Chrome):
    # Add Assessment Purpose
    text = extract_text(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtQuantity")
    print(f"Extracted Text is {text}")
    write_text(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtQuantity_Statistical_Purpose",
               text=text)
    # write_text(driver=driver,id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtQuantity_International_Traded",text=text)
    # Now Click
    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_btnCalcExportValue")
    time.sleep(3)
    # write_text(driver=driver,id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtActualWeight",text="1002.0000")


def populate_username_passowrd_in_login_form(driver, transaction_id, url, username, password):
    status = False
    error = ''

    try:
        driver.get(url)

        username_input = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[placeholder='Enter Username']"))
        )
        password_input = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[placeholder='Enter Password']"))
        )

        username_input.send_keys(username)
        password_input.send_keys(password)
        sign_in_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Login')]")
        sign_in_button.click()
        status = True
    except TimeoutException:
        error = f"For transaction_id {transaction_id} Login Page did not Appear."
    except Exception as e:
        error = f"For transaction_id {transaction_id} Error Occured => {str(e)}"
    return status, error


def select_types(driver, transaction_id):
    Create_Export_GD = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, "//a[contains(., 'Create Export GD')]"))
    )
    print(f"Found Create_Export_GD.")
    # Perform an action on the input element (e.g., click)
    Create_Export_GD.click()
    select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_GdSelectionExport1_ddlConsignmentType",
                    option_text="Export Facilitation Scheme")
    select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_GdSelectionExport1_ddlDeclarationType",
                    option_text="Export Facilitation Scheme")

    Create_button = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, "(//input[@value='Create'])[2]"))
    )
    print(f"Found Create_button.")
    # Perform an action on the input element (e.g., click)
    Create_button.click()
    print(f"Clicked Create_button.")


def select_saved(driver, transaction_id):
    saved_GD = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_lnkExportSaved"))
    )
    print(f"Found saved_GD.")
    saved_GD.click()
    GD = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_dgSaved_ctl02_lbTrackingId"))
    )
    print(f"Found GD.")
    GD.click()
    GD = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_btnSaveTop"))
    )
    print("GD ready")


def select_GDS(driver, transaction_id):
    status = False
    error = ''
    print(f"For transaction_id {transaction_id} Selecting GDS...")
    try:
        traders_button = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='List of Traders'])[2]"))
        )
        print(f"Found traders_button.")
        traders_button.click()
        iframe = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
        )
        driver.switch_to.frame(iframe)
        print(f"iframe Switched.")
        time.sleep(3)
        company_element = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located(
                (By.XPATH, "//td[text()='STYLE TEXTILE (PRIVATE) LTD']/preceding-sibling::td[1]//input[@type='image']"))
        )
        print(f"Found company_element.")
        # Perform an action on the input element (e.g., click)
        company_element.click()
        driver.switch_to.default_content()
        time.sleep(3)
        print(f"Switched to default_content.")
        goods_declaration = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='Goods Declaration'])[2]"))
        )
        print(f"Found goods_declaration.")
        goods_declaration.click()
        driver.switch_to.frame(iframe)
        select_types(driver, transaction_id)
        # select_saved(driver,transaction_id)
        status = True
    except Exception as e:
        error = f"For transaction_id {transaction_id} Error Occured => {str(e)}"
        print(e)
    return status, error


def fill_form(driver: webdriver.Chrome, transaction_id, data={}):
    status = False
    error = ''
    pl_data = data.get("pl_data")
    fty_data = data.get("fty_data")
    print(f"For transaction_id {transaction_id} Filling the Form...")
    try:
        # Basic Info
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_BasicInfoUc1_ddlCollectorate",
                        option_text="Port Qasim (exports), karachi")
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_BasicInfoUc1_ddlModeOfTransport",
                        option_text="Inland water Transport")

        # Consignor & Consignee Information (Hide)
        write_text(driver, "ctl00_ContentPlaceHolder2_ConsigneeInfoUc1_txtConsignorName", pl_data.get("consignee"))
        write_text(driver, "ctl00_ContentPlaceHolder2_ConsigneeInfoUc1_txtConsignorAddress", pl_data.get("address"))

        # GD Information
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_GdInfoSeaUc_ddlConsignmentMode",
                        option_text="Part Shipment")
        # SET the BL/AWB No (Invoice No Last Six digits)
        invoices = data.get("final_table").get('invoices')
        bl_awb_no = invoices.split("\n")[0]
        write_text(driver, "ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtBlNo",
                   bl_awb_no)
        #  SET the BL/AWB Date (Invoice Date)
        # Parse the date string into a datetime object
        date_object = datetime.strptime(fty_data.get("extracted_data")[0].get('invoice_date'), "%Y-%m-%d")
        write_date(driver, "ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtBlDate_txtDate", date_object.strftime("%d/%m/%Y"))
        #  Select the Port of Shipment dropdown
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_GdInfoSeaUc_ddlPortOfShipment",
                        option_text="Port Qasim (exports), karachi")
        #  Select the Destination Country
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_GdInfoSeaUc_ddlDestinationCountry",
                        option_text="United States")
        #  Select the Destination Port
        select_dropdown_by_value(driver=driver, id="ctl00_ContentPlaceHolder2_GdInfoSeaUc_ddlPortOfDischarge",
                                 value="41846")
        # Select the Shipping Line
        select_dropdown_by_value(driver=driver, id="ctl00_ContentPlaceHolder2_GdInfoSeaUc_ddlShippingline", value="66")
        # SET Place of Delivery
        write_text(driver, "ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtPlaceofDelivery", "Long Beach, USA")
        # SET Net Weight (MT)
        net_weight = data.get("final_table").get("Net Weight") / 1000
        write_text(driver, "ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtNetWeight", net_weight)
        # SET Gross Weight
        gross_weight = data.get("final_table").get("Gross Weight") / 1000
        write_text(driver, "ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtGrossWeight", gross_weight)
        # SET Marks
        marks = f"""AS PER SHIPPER \n INVOICE NO.\n{invoices}"""
        write_text(driver, "ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtMarks", marks)

        # financials_info
        # Select the PAYMENT TERM dropdown
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_FinancialsUc1_ddlPaymentTerms",
                        option_text="Without LC")
        # Select the Bank dropdown
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_FinancialsUc1_ddlBank",
                        option_text="MCB BANK LIMITED")
        # Select the Delivery Term dropdown
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_FinancialsUc1_ddlDeliveryTerm",
                        option_text="Free On Board (FOB)")
        # Select the Currency dropdown
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_FinancialsUc1_ddlCurrency",
                        option_text="United States-US $")
        # SET FOB Value
        write_text(driver, "ctl00_ContentPlaceHolder2_FinancialsUc1_txtFobValue",
                   data.get("final_table").get("PO Net Amount"))

        # supporting_info_fill

        # Select the Shed/Location Code dropdown
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_SupportingInfoUc1_ddlExaminerGroup",
                        option_text="IC3")
        # Select the ddlTerminal dropdown
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_SupportingInfoUc1_ddlTerminal",
                        option_text="Qasim International Container Terminal")

        # check_disclaimer
        checkbox(driver, id="ctl00_ContentPlaceHolder2_ExportPolicyDisclaimerUc1_chkExportPolicy")
        checkbox(driver, id="ctl00_ContentPlaceHolder2_ExportPolicyDisclaimerUc1_chkIsSroDisclaimer")

        # Click the Save button
        click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnSaveBottom")

        print(f"GD Filling Success")
    except Exception as e:
        error = f"For transaction_id {transaction_id} Error Occured => {str(e)}"
        print(e)
    return status, error


def upload_document(driver: webdriver.Chrome, filepath: os.path.join):
    UploadButton = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH,
                                        "//a[@id='ctl00_ContentPlaceHolder2_UploadDocumentUc1_lnkUploadDocument' and text()='Upload Document']"))
    )
    print(f"Found Upload Button.")
    UploadButton.click()
    fileButton = WebDriverWait(driver, 1000).until(
        EC.presence_of_element_located((By.XPATH,
                                        "//table[.//span[text()='Attached Document']]//input[@id='ctl00_ContentPlaceHolder2_GdExportUploadDocUc1_GdImportDocUpload_fuDoc']"))
    )
    print("fileButton ready")
    fileButton.send_keys(filepath)
    if 'fty' in str(filepath).lower():
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_GdExportUploadDocUc1_cmbDocumentType",
                        option_text="Invoice")
    else:
        select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_GdExportUploadDocUc1_cmbDocumentType",
                        option_text="Packing List")
    UploadDocButton = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH,
                                        "//table[.//span[text()='Attached Document']]//input[@id='ctl00_ContentPlaceHolder2_GdExportUploadDocUc1_btnUpload' and @value='Upload']"))
    )
    print(f"Found Upload DOC Button.")
    UploadDocButton.click()

    WebDriverWait(driver, 500).until(
        EC.presence_of_element_located((By.XPATH,
                                        "//table[.//span[text()='Customs Office']]//select[@id='ctl00_ContentPlaceHolder2_BasicInfoUc1_ddlCollectorate']"))
    )
    print("GD ready")


def upload_documents(driver: webdriver.Chrome, pdf_path):
    for file in pdf_path:
        print(f"Uploading file: {file}")
        upload_document(driver, file)


def fill_container_info(driver: webdriver.Chrome, data={}):
    # Now Click on Container Info

    click_button(driver=driver,
                 id="//table[.//span[text()='Containers Information']]//a[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgContainer_ctl02_lnkBtnAdd']",
                 by=By.XPATH)

    write_text(driver=driver,
               id="//table[.//span[text()='Containers Information']]//input[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgContainer_ctl02_txtContainerNo']",
               text="STYLE123456", by=By.XPATH)
    quantity = str(data.get('Quantity'))
    write_text(driver=driver,
               id="//table[.//span[text()='Containers Information']]//input[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgContainer_ctl02_txtQuantity']",
               text=quantity, by=By.XPATH)
    cartons = str(data.get('Carton'))
    write_text(driver=driver,
               id="//table[.//span[text()='Containers Information']]//input[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgContainer_ctl02_txtNoOfPackages']",
               text=cartons, by=By.XPATH)
    select_dropdown(driver=driver,
                    id="//table[.//span[text()='Containers Information']]//select[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgContainer_ctl02_ddlPackageType']",
                    option_text="CARTONS", by=By.XPATH)
    click_button(driver=driver,
                 id="//table[.//span[text()='Containers Information']]//a[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_dgContainer_ctl02_lnkBtnAdd']",
                 by=By.XPATH)


def add_item(driver: webdriver.Chrome, transaction_id, data={},item_no=None):
    print(f"For transaction_id {transaction_id} Adding Item...")
    click_button(driver=driver, id="//a[@id='ctl00_ContentPlaceHolder2_ItemInfoUc1_lnkItems' and text()='Add Items']",
                 by=By.XPATH)
    print("Clicked Add Items")
    write_text(driver, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtHsCode", data.get('hs_code'))
    description = data.get('description')
    if item_no:
        additional_text = "UNDER CLAIM FOR 'DRAWBACK' OF LOCAL TAXES AND BRAND NOTIFICATION NO. 3(1) TID/09-P-I DATED: 01.09.2009 IMPORTED MATERIAL USED UNDER EFS SRO 957(I)21 DT.30.07.2021) (DETAILS AS PER INVOICE) INVOICE NO."
        description = f"{description} \n {additional_text} {data.get('invoice_number')}"
    else:
        additional_text = "(DETAILS AS PER INVOICE) INVOICE NO. "
        description = f"{description} \n {additional_text} {data.get('invoice_number')}"
    write_text(driver, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtDeclaredDescription", description)
    select_dropdown_by_value(driver, 'ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_ddlOrigion', '586')
    unit_value = (data.get('PO Net Amount')) / (data.get('Quantity'))
    write_text(driver, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtUnitValue", unit_value)
    write_text(driver, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtActualWeight", data.get('Quantity'))
    # Select Actual Unit now
    select_dropdown(driver=driver, id="ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_ddlActualWeightUnit",
                    option_text="NO")
    print("filling the exemptions")
    exemptions(driver)
    # Check	BLEACHED BLENDED GARMENTS, WEARING APPAREL (ALL BLENDS OF POLYESTER STAPLE FIBRE AND COTTON FIBRE).
    process_duty_drawback(driver, data.get('description',''))
    # Now Handel Quantity (for Assessment Purpose)
    time.sleep(5)
    WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH,
                                        "//table[.//span[text()='HS Code']]//input[@id='ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_txtHsCode']"))
    )
    print(f"Now Executing assessment_purpose")

    fill_container_info(driver=driver, data=data)
    assessment_purpose(driver)
    # Now Click on Save Button
    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnSaveBottom")
    WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_BasicInfoUc1_pnlTitle"))
    )
    print("HScode Added")
    return data.get('hs_code')


def process_gd_number_pop_up_492(driver : webdriver.Chrome,data):
            # Store the ID of the original window
    original_window = driver.current_window_handle

    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_btnGDLookup")

    # Wait for the new window or tab (assume we know a new window opens here)
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Get the list of all window handles
    windows = driver.window_handles

    # Switch to the new window
    for window in windows:
        if window != original_window:
            driver.switch_to.window(window)
            break

    # Now you can interact with the new window
    # For example, finding an element and interacting with it
    write_text(driver, "txtSearch",data.get('B/E No'),pop_up=True)
    click_button(driver, "btnSearch",pop_up=True)
    time.sleep(5)
    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "tblAlert"))
    )
    if 'no data found' in table.text.lower():
        try:
            driver.close()
            driver.switch_to.window(original_window)
            iframe = WebDriverWait(driver, 100).until(
                    EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
                )
            driver.switch_to.frame(iframe)
        except NoSuchWindowException:
            print("The new window was already closed.")
        return None
    click_button(driver, id="//tr[@class='ItemStyle']//a[@id='dgLookup_ctl02_lbSelect']",by=By.XPATH,pop_up=True)
        # Attempt to close the new window
    try:
        driver.close()
    except NoSuchWindowException:
        print("The new window was already closed.")

    # Switch back to the original window
    driver.switch_to.window(original_window)
    iframe = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
        )
    driver.switch_to.frame(iframe)
    # Locate the table
    table = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_dgItems"))
        )

    # Find all rows in the table body
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Iterate through the rows, skipping the header
    for row in rows[1:]:
        cells = row.find_elements(By.TAG_NAME, "td")
        if cells:
            try:
                unit_value = float(cells[4].text)
                puv = float(data.get('PER UNIT VALUE'))
                if math.isclose(unit_value, puv, rel_tol=0.1):
                    select_link = cells[0].find_element(By.TAG_NAME, "a")
                    if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
                        select_link.click()
                    else:
                        return None
                    time.sleep(2)
                    break
            except ValueError:
                print(f"Skipping row due to non-numeric value: {cells[4].text}")
    else:
        # select 1st row
        row = rows[1]
        cells = row.find_elements(By.TAG_NAME, "td")
        if cells:
            select_link = cells[0].find_element(By.TAG_NAME, "a")
            print(f"No matching row found in the table for PER UNIT VALUE {data.get('PER UNIT VALUE')} and {data.get('B/E No')} Selecting 1st row")
            if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
                select_link.click()
            else:
                print("Select Link is not enabled")
                return None
        else:
            return None
    try:
        Quantity = float(extract_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtQuantity"))
    except :
        Quantity = 0.0
    try:
        CONSUMED = float(data.get('NOW CONSUMED'))
    except :
        CONSUMED = 0.0
    if CONSUMED < Quantity:
        write_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtQuantity",data.get('Now Consume'),pop_up=True)
    return True

def process_gd_number_pop_up_957(driver : webdriver.Chrome,data,is_hscode_wise=False):
    be_no = 'B/E No/PACKAGE NO/PURCHASE INV#'
    if is_hscode_wise:
        be_no = 'B/E No'
            # Store the ID of the original window
    original_window = driver.current_window_handle

    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_btnGDLookup")

    # Wait for the new window or tab (assume we know a new window opens here)
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Get the list of all window handles
    windows = driver.window_handles

    # Switch to the new window
    for window in windows:
        if window != original_window:
            driver.switch_to.window(window)
            break

    # Now you can interact with the new window
    # For example, finding an element and interacting with it
    write_text(driver, "txtSearch",data.get(be_no),pop_up=True)
    click_button(driver, "btnSearch",pop_up=True)
    time.sleep(5)
    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "tblAlert"))
    )
    if 'no data found' in table.text.lower():
        try:
            driver.close()
            driver.switch_to.window(original_window)
            iframe = WebDriverWait(driver, 100).until(
                    EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
                )
            driver.switch_to.frame(iframe)
        except NoSuchWindowException:
            print("The new window was already closed.")
        return None
    click_button(driver, id="//tr[@class='ItemStyle']//a[@id='dgLookup_ctl02_lbSelect']",by=By.XPATH,pop_up=True)
        # Attempt to close the new window
    try:
        driver.close()
    except NoSuchWindowException:
        print("The new window was already closed.")

    # Switch back to the original window
    driver.switch_to.window(original_window)
    iframe = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
        )
    driver.switch_to.frame(iframe)
    # Locate the table
    table = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_dgItems"))
        )

    # Find all rows in the table body
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Iterate through the rows, skipping the header
    for row in rows[1:]:
        cells = row.find_elements(By.TAG_NAME, "td")
        if cells:
            print("The celss text is : ",cells[4].text)
            try:
                unit_value = float(cells[4].text)
                puv = float(data.get('PER UNIT VALUE'))
                if math.isclose(unit_value, puv, rel_tol=0.1):
                    select_link = cells[0].find_element(By.TAG_NAME, "a")
                    if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
                        select_link.click()
                    else:
                        return None
                    time.sleep(2)
                    break
            except ValueError:
                print(f"Skipping row due to non-numeric value: {cells[4].text}")
    else:
        print('In rows')
        # select 1st row
        row = rows[1]
        cells = row.find_elements(By.TAG_NAME, "td")
        if cells:
            select_link = cells[0].find_element(By.TAG_NAME, "a")
            print(f"No matching row found in the table for PER UNIT VALUE {data.get('PER UNIT VALUE')} and {data.get(be_no)} Selecting 1st row")
            if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
                select_link.click()
                time.sleep(5)
            else:
                print("Select Link is not enabled")
                return None
        else:
            return None
    try:
        Quantity = float(extract_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtQuantity"))
    except :
        Quantity = 0.0
    try:
        CONSUMED = float(data.get('NOW CONSUMED'))
    except :
        CONSUMED = 0.0
    if CONSUMED < Quantity:
        write_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtQuantity",data.get('NOW CONSUMED'),pop_up=True)
    hs_code = extract_text(driver, "ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_txtHsCode")
    return hs_code

def process_analysis_number_pop_up_957(driver : webdriver.Chrome,analysis_number,hs_code):
            # Store the ID of the original window
    original_window = driver.current_window_handle

    click_button(driver=driver,id="ctl00_ContentPlaceHolder2_NonDutyPaidItemDetailUc1_btnAnalysisLookup")

    # Wait for the new window or tab (assume we know a new window opens here)
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Get the list of all window handles
    windows = driver.window_handles

    # Switch to the new window
    for window in windows:
        if window != original_window:
            driver.switch_to.window(window)
            break

    # Now you can interact with the new window
    # For example, finding an element and interacting with it
    write_text(driver, "txtSearch",analysis_number,pop_up=True)
    write_text(driver, "txtInputHSCode",hs_code,pop_up=True)
    click_button(driver, "btnSearch",pop_up=True)
    time.sleep(3)
    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "tblAlert"))
    )
    if 'no data found' in table.text.lower():
        try:
            driver.close()
            driver.switch_to.window(original_window)
            iframe = WebDriverWait(driver, 100).until(
                    EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
                )
            driver.switch_to.frame(iframe)
        except NoSuchWindowException:
            print("The new window was already closed.")
        return None
    click_button(driver, id="//tr[@class='ItemStyle']//a[@id='dgLookupExport_ctl02_lbSelect']",by=By.XPATH,pop_up=True)
        # Attempt to close the new window
    try:
        driver.close()
    except NoSuchWindowException:
        print("The new window was already closed.")

    # Switch back to the original window
    driver.switch_to.window(original_window)
    iframe = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
        )
    driver.switch_to.frame(iframe)
    time.sleep(2)
    wait_for_page_load(driver)


def add_excel_data_492(driver: webdriver.Chrome, data):
    # Wait until the image is present
    try:
        toggle_NonDutyPaid(driver)
        click_button(driver=driver,
                     id="//a[@id='ctl00_ContentPlaceHolder2_NonDutyPaidItemInfoUc1_lnkItems' and text()='Attach Item']",
                     by=By.XPATH)
        pop_up_492 = process_gd_number_pop_up_492(driver, data)
        if pop_up_492:
            click_button(driver=driver, id="//input[@id='ctl00_ContentPlaceHolder2_btnSaveBottom']", by=By.XPATH)

            WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_btnSaveTop"))
            )
            print(f"Element in pop up Added.")
        else:
            print(f"HS Code Not Found for {data.get('B/E No')}")
            # Cancel the present filling
            click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnCancelBottom")
    except:
        print(f"Got Error for {data.get('B/E No')}")
        # Cancel the present filling
        click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnCancelBottom")
        WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_pnlTitle"))
        )


def add_excel_data_957(driver: webdriver.Chrome, data, analysis_number, is_hscode_wise=False):
    be_no = 'B/E No/PACKAGE NO/PURCHASE INV#'
    if is_hscode_wise:
        be_no = 'B/E No'
    toggle_NonDutyPaid(driver)
    click_button(driver=driver,
                 id="//a[@id='ctl00_ContentPlaceHolder2_NonDutyPaidItemInfoUc1_lnkItems' and text()='Attach Item']",
                 by=By.XPATH)
    try:
        hs_code = process_gd_number_pop_up_957(driver, data, is_hscode_wise)
        if hs_code:
            process_analysis_number_pop_up_957(driver, analysis_number=analysis_number, hs_code=hs_code)
            click_button(driver=driver, id="//input[@id='ctl00_ContentPlaceHolder2_btnSaveBottom']", by=By.XPATH)

            WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_pnlTitle"))
            )
            print(f"Element in pop up Added.")
        else:
            print(f"HS Code Not Found for {data.get(be_no)}")
            # Cancel the present filling
            click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnCancelBottom")
    except Exception as e:
        print(f"Got Error for {data.get(be_no)} => {str(e)}")
        # Cancel the present filling
        click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnCancelBottom")
        WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_pnlTitle"))
        )


def process_492(driver, data):
    for idx, obj in enumerate(data):
        if obj.get('B/E No'):
            add_excel_data_492(driver, data=obj)


def process_957(driver, data, analysis_number, is_hscode_wise=False):
    be_no = 'B/E No/PACKAGE NO/PURCHASE INV#'
    if is_hscode_wise:
        be_no = 'B/E No'
    for obj in data:
        if categorize_invoice(obj.get(be_no)) == 'non_local':
            add_excel_data_957(driver, data=obj, analysis_number=analysis_number, is_hscode_wise=is_hscode_wise)
        else:
            print(f"Local Invoice {obj.get(be_no)}")
            add_excel_data_local(driver, data=obj, analysis_number=analysis_number, is_hscode_wise=is_hscode_wise)


def process_localy_purchased_analysis_no_pop_up_957(driver: webdriver.Chrome, analysis_number, hs_code):
    # Store the ID of the original window
    original_window = driver.current_window_handle

    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_LocalPurchaseItemEntryUc1_btnAnalysisLookup")

    # Wait for the new window or tab (assume we know a new window opens here)
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Get the list of all window handles
    windows = driver.window_handles

    # Switch to the new window
    for window in windows:
        if window != original_window:
            driver.switch_to.window(window)
            break

    # Now you can interact with the new window
    # For example, finding an element and interacting with it
    write_text(driver, "txtSearch", analysis_number, pop_up=True)
    write_text(driver, "txtInputHSCode", hs_code, pop_up=True)
    click_button(driver, "btnSearch", pop_up=True)
    time.sleep(3)
    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "tblAlert"))
    )
    if 'no data found' in table.text.lower():
        try:
            driver.close()
            driver.switch_to.window(original_window)
            iframe = WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
            )
            driver.switch_to.frame(iframe)
        except NoSuchWindowException:
            print("The new window was already closed.")
        return None
    click_button(driver, id="//tr[@class='ItemStyle']//a[@id='dgLookupExport_ctl02_lbSelect']", by=By.XPATH,
                 pop_up=True)
    # Attempt to close the new window
    try:
        driver.close()
    except NoSuchWindowException:
        print("The new window was already closed.")

    # Switch back to the original window
    driver.switch_to.window(original_window)
    iframe = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
    )
    driver.switch_to.frame(iframe)
    wait_for_page_load(driver)
    return True


def process_localy_purchased_pop_up_957(driver: webdriver.Chrome, data, is_hscode_wise=False):
    be_no = 'B/E No/PACKAGE NO/PURCHASE INV#'
    if is_hscode_wise:
        be_no = 'B/E No'
        # Store the ID of the original window
    original_window = driver.current_window_handle

    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_LocalPurchaseItemEntryUc1_btnLocalPurchaseLookup")

    # Wait for the new window or tab (assume we know a new window opens here)
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Get the list of all window handles
    windows = driver.window_handles

    # Switch to the new window
    for window in windows:
        if window != original_window:
            driver.switch_to.window(window)
            break

    # Now you can interact with the new window
    # For example, finding an element and interacting with it
    write_text(driver, "txtSearch", data.get(be_no), pop_up=True)
    click_button(driver, "btnSearch", pop_up=True)
    time.sleep(5)
    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "tblAlert"))
    )
    if 'no data found' in table.text.lower():
        try:
            driver.close()
            driver.switch_to.window(original_window)
            iframe = WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
            )
            driver.switch_to.frame(iframe)
        except NoSuchWindowException:
            print("The new window was already closed.")
        return None
    click_button(driver, id="//tr[@class='ItemStyle']//a[@id='dgLookup_ctl02_lbSelect']", by=By.XPATH, pop_up=True)
    # Attempt to close the new window
    try:
        driver.close()
    except NoSuchWindowException:
        print("The new window was already closed.")

    # Switch back to the original window
    driver.switch_to.window(original_window)
    iframe = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, "(//*[@id='frame'])[1]"))
    )
    driver.switch_to.frame(iframe)
    # Locate the table

    table = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_LocalPurchaseItemEntryUc1_dgItems"))
    )

    # Find all rows in the table body
    rows = table.find_elements(By.TAG_NAME, "tr")

    row = rows[1]
    cells = row.find_elements(By.TAG_NAME, "td")
    if cells:

        select_link = cells[0].find_element(By.TAG_NAME, "a")
        print(
            f"No matching row found in the table for PER UNIT VALUE {data.get('PER UNIT VALUE')} and {data.get(be_no)} Selecting 1st row")
        if select_link.is_enabled() and select_link.get_attribute("disabled") is None:
            select_link.click()
            time.sleep(2)
        else:
            print("Select Link is not enabled")
            return None
    else:
        return None
    hs_code = extract_text(driver, "ctl00_ContentPlaceHolder2_LocalPurchaseItemEntryUc1_txtHsCode")
    return hs_code


def add_excel_data_local(driver: webdriver.Chrome, data, analysis_number, is_hscode_wise=False):
    be_no = 'B/E No/PACKAGE NO/PURCHASE INV#'
    if is_hscode_wise:
        be_no = 'B/E No'
    # Wait until the image is present
    toggle_LocalPurchaseItem(driver)
    click_button(driver=driver,
                 id="//a[@id='ctl00_ContentPlaceHolder2_LocalPurchaseItemEntryInfoUc1_lnkItems' and text()='Attach Locally Purchase Item']",
                 by=By.XPATH)
    time.sleep(2)

    hs_code = process_localy_purchased_pop_up_957(driver, data, is_hscode_wise)
    if hs_code:
        res_957 = process_localy_purchased_analysis_no_pop_up_957(driver, analysis_number=analysis_number,
                                                                  hs_code=hs_code)
        if res_957:
            click_button(driver=driver, id="//input[@id='ctl00_ContentPlaceHolder2_btnSaveBottom']", by=By.XPATH)
            time.sleep(1)
        else:
            # Cancel the present filling
            click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnCancelBottom")
            time.sleep(1)
        WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_LocalPurchaseItemEntryInfoUc1_lblTitle"))
        )
        print(f"Element in pop up Added.")
    else:
        print(f"HS Code Not Found for {data.get(be_no)}")
        # Cancel the present filling
        click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnCancelBottom")


def Non_Duty_Paid_Info(driver, csv_obj: CSVDataExtractor, hs_code, elem_index):
    select_added_item(driver)
    # Wait until the image is present

    data_492 = csv_obj.table492_data
    data_957 = csv_obj.table957_data

    process_492(driver, data_492)
    analysis_number = csv_obj.get_analysis_number(hs_code)
    process_957(driver, data_957, analysis_number)
    time.sleep(5)
    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnSaveBottom")
    WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_BasicInfoUc1_pnlTitle"))
    )


def Non_Duty_Paid_Info_multi_po(driver, csv_obj: CSVDataExtractor, hs_code, elem_index):
    select_added_item(driver)
    toggle_NonDutyPaid(driver)

    data_957_hs_code_wise = csv_obj.hs_code_wise_tables.get(hs_code)
    data_957 = data_957_hs_code_wise.get('sub_table')

    analysis_number = csv_obj.get_analysis_number(hs_code)
    process_957(driver, data_957, analysis_number, is_hscode_wise=True)
    time.sleep(5)
    click_button(driver=driver, id="ctl00_ContentPlaceHolder2_btnSaveBottom")
    WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_BasicInfoUc1_pnlTitle"))
    )


def process_multi_single(driver, items_data, prev_idx=0):
    for idx, item_data in enumerate(items_data):
        item_no=True if idx == 0 else None
        hs_code = add_item(driver, 1, data=item_data, item_no=item_no)
        Non_Duty_Paid_Info(driver, item_data.get('csv_obj'), hs_code, elem_index=idx + 1 + prev_idx)
        print(f"GD Completed For Item : {item_data}")
    return idx


def process_multi_po(driver, po_obj: MultiPOParse):
    po_tables = po_obj.extracted_data.get('po_tables')
    invoice_nos = po_tables.keys()
    print(invoice_nos)
    for invoice in invoice_nos:
        items_data = po_tables[invoice].get('po_numbers')
        print(items_data)
        for idx, item_data in enumerate(items_data):

            data_to_send = po_obj.get_item_info(int(item_data))
            totals = po_tables[invoice].get('totals')
            csv_obj = po_tables[invoice].get('csv_obj')
            data_to_send['invoice_number'] = str(invoice)
            data_to_send['csv_obj'] = csv_obj
            data_to_send.update(totals)
            data_to_send['Carton'] = data_to_send['CTNS']
            data_to_send['Quantity'] = data_to_send['PCS']
            print(data_to_send)
            if not data_to_send.get('hs_code'):
                continue
            else:
                if idx == 0:
                    add_item(driver, 1, data=data_to_send,item_no=True)
                else:
                    add_item(driver, 1, data=data_to_send)

                if idx == 0:
                    Non_Duty_Paid_Info(driver, data_to_send.get('csv_obj'), hs_code=data_to_send.get('hs_code'),
                                       elem_index=idx + 1)
                    print(f"Adding HS Code Wise Tables for hs code {data_to_send.get('hs_code')}")
                    Non_Duty_Paid_Info_multi_po(driver, data_to_send.get('csv_obj'),
                                                hs_code=data_to_send.get('hs_code'), elem_index=idx + 1)
                    print(f"GD Completed For Item : {item_data}")
                else:
                    Non_Duty_Paid_Info_multi_po(driver, data_to_send.get('csv_obj'),
                                                hs_code=data_to_send.get('hs_code'), elem_index=idx + 1)

    return idx
def main(data):
    try:
        print('Starting the process')
        finalStatus = False
        finalMessage =''
        driver = setup_driver()
        transaction_id = data.get("transaction_id")
        username = data["UserName"]
        passwd = data["Password"]
        url = data["URL"]
        login_form_status = True
        login_form_status, login_form_error = populate_username_passowrd_in_login_form(driver, transaction_id,url, username, passwd)
        if login_form_status:
            # pickle.dump(driver.get_cookies(), open("cookies.pkl", "wb"))
            print(f"For transaction_id { transaction_id } Cookies Done...")
            wait = WebDriverWait(driver, 60)
            # Define a condition to wait for page load (you can customize this)
            page_load_condition = EC.presence_of_element_located((By.ID, "sidebar-container"))
            # Wait until the page is fully loaded
            wait.until(page_load_condition)
            time.sleep(3)
            GD_status, GD_error = select_GDS(driver, transaction_id)
            print(f"GD_status: {GD_status}, GD_error: {GD_error}")
            if GD_status:
                fill_form_status, fill_form_error = fill_form(driver, transaction_id,data=data)
                time.sleep(5)
                print(f"Uploading the documents")
                upload_documents(driver,pdf_path=data.get('pdf_paths'))
                time.sleep(5)
                fty_data = data.get('fty_data')
                po_obj = data.get('po_obj')
                
                prev_idx = 0
                print(f"Starting the Multi PO Process")
                if po_obj:
                    print(f"Starting the Multi PO Process")
                    prev_idx = process_multi_po(driver,po_obj)
                if fty_data:
                    items_data = fty_data.get('extracted_data')
                    print(f"Now Starting the Multi Single Process")
                    process_multi_single(driver,items_data,prev_idx)
        else:
            finalMessage = login_form_error

    except Exception as e:
        print(f"For transaction_id { transaction_id } Exception occurred: {e}")
        finalStatus = False
        finalMessage = str(e)
    finally:
        return driver
pl_pdf_path, fty_pdf_path, csv_path = extract_files_club_single()
pl_po_pdf_path, fty_po_pdf_path, csv_po_path,desc_po_path = extract_files_club_po()

fty_parser = MultiSingleParse(fty_pdf_path,csv_path=csv_path)
pl_parser = PlParse(pl_po_pdf_path)
po_parser = MultiPOParse(path=fty_po_pdf_path,csv_path=csv_po_path,des_path=desc_po_path)
pdf_paths = [pl_pdf_path, fty_pdf_path]
final_table_data = add_data_dictionaries(po_parser.extracted_data['final_table'],fty_parser.extracted_data['final_table'])

data = {
    'transaction_id': 1,
    'user_id': 2,
    'URL': 'https://app.psw.gov.pk/app/',
    'UserName': 'CA-01-2688539',
    'Password': 'Express@3833',
    'pdf_paths': pdf_paths,
    'pl_data':pl_parser.extracted_data,
    'fty_data':fty_parser.extracted_data,
    'po_obj':po_parser,
    'final_table':final_table_data,
}
print(fty_parser.extracted_data['extracted_data'][0].get('csv_obj').table492_data)
print(fty_parser.extracted_data['extracted_data'][0].get('csv_obj').table957_data)
print(fty_parser.extracted_data['extracted_data'][0].get('csv_obj').table1_data)
required_keys = list(data.keys())

driver = main(data)


PL PDF Path: c:\Users\DELL\Downloads\PWSAutomate\multi\multisingle\pl.pdf
FTY PDF Path: c:\Users\DELL\Downloads\PWSAutomate\multi\multisingle\fty.pdf
CSV Path: c:\Users\DELL\Downloads\PWSAutomate\multi\multisingle
PL PDF Path: c:\Users\DELL\Downloads\PWSAutomate\multi\multipos\pl.pdf
FTY PDF Path: c:\Users\DELL\Downloads\PWSAutomate\multi\multipos\fty3.pdf
CSV Path: c:\Users\DELL\Downloads\PWSAutomate\multi\multipos
adidas branded garments. export from export facilitation scheme (efs) 134740069 unisex infants 78% cotton 22% recycled polyesterknit pullover dyed
Women 80/20% Pullovers
Searching for CSV file for invoice number 507944 in c:\Users\DELL\Downloads\PWSAutomate\multi\multisingle...
EFS507944.csv
adidas branded garments. export from export facilitation scheme (efs) 134743455 womens 70% cotton 30% recycle polyster knit t-shirts dyed
Women 80/20% T-Shirts
Searching for CSV file for invoice number 507957 in c:\Users\DELL\Downloads\PWSAutomate\multi\multisingle...
EFS507944.csv
EFS5

NameError: name 'fty_parser' is not defined

In [1]:
def find_hs_code(description):
    description = description.lower().strip().replace('\n', '')
    print(description)
    material_category = determine_category(description)

    garment_type = None
    gender_category = None

    if 't-shirt' in description or 't shirt' in description:
        garment_type = 'T-Shirts'
    elif 'shirt' in description:
        garment_type = 'Shirts'
    elif 'pant' in description or 'shorts' in description:
        garment_type = 'Pants'
    elif 'jacket' in description:
        garment_type = 'Jackets'
    elif 'track suit' in description:
        garment_type = 'Track Suits'
    elif 'pullover' in description:
        garment_type = 'Pullovers'

    if 'unisex' in description:
        gender_category = 'Women'
    elif 'women' in description or 'ladies' in description or 'girl' in description:
        gender_category = 'Women'
    elif 'men' in description or 'gentle' in description or 'boy' in description or 'kid' in description:
        gender_category = 'Men'
    
    print(f"{gender_category} {material_category} {garment_type}")
    if material_category and garment_type and gender_category:
        key = f"{gender_category} {material_category} {garment_type}"
        return hs_codes.get(key)

    return None

def process_multi_po(driver ,po_obj:MultiPOParse):
    po_tables = po_obj.extracted_data.get('po_tables')
    invoice_nos = po_tables.keys()
    print(invoice_nos)
    for invoice in invoice_nos:
        
        items_data = po_tables[invoice].get('po_numbers')
        print(items_data)
        for idx,item_data in enumerate(items_data):
            if idx == 0:
                continue
            else:
                data_to_send = po_obj.get_item_info(int(item_data))
                totals = po_tables[invoice].get('totals')
                csv_obj = po_tables[invoice].get('csv_obj')
                data_to_send['csv_obj'] = csv_obj
                data_to_send.update(totals)
                data_to_send['Carton'] = data_to_send['CTNS']
                data_to_send['Quantity'] = data_to_send['PCS']
                print(data_to_send)
                # if not data_to_send.get('hs_code'):
                #     continue
                # else:
                #     add_item(driver, 1,data=data_to_send)
                    
                #     if idx == 0:
                #         # Non_Duty_Paid_Info(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
                #         print(f"Adding HS Code Wise Tables for hs code {data_to_send.get('hs_code')}")
                #         Non_Duty_Paid_Info_multi_po(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
                #         print(f"GD Completed For Item : {item_data}")
                #     else:
                #         Non_Duty_Paid_Info_multi_po(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
            
    return idx
process_multi_po(driver=driver,po_obj=po_parser)
po_parser.po_data

NameError: name 'MultiPOParse' is not defined

In [ ]:
driver = main(data)

In [ ]:
def find_hs_code(description):
    description = description.lower().strip().replace('\n', '')
    print(description)
    material_category = determine_category(description)

    garment_type = None
    gender_category = None

    if 't-shirt' in description or 't shirt' in description:
        garment_type = 'T-Shirts'
    elif 'shirt' in description:
        garment_type = 'Shirts'
    elif 'pant' in description or 'shorts' in description:
        garment_type = 'Pants'
    elif 'jacket' in description:
        garment_type = 'Jackets'
    elif 'track suit' in description:
        garment_type = 'Track Suits'
    elif 'pullover' in description:
        garment_type = 'Pullovers'

    if 'unisex' in description:
        gender_category = 'Women'
    elif 'women' in description or 'ladies' in description or 'girl' in description:
        gender_category = 'Women'
    elif 'men' in description or 'gentle' in description or 'boy' in description or 'kid' in description:
        gender_category = 'Men'
    
    print(f"{gender_category} {material_category} {garment_type}")
    if material_category and garment_type and gender_category:
        key = f"{gender_category} {material_category} {garment_type}"
        return hs_codes.get(key)

    return None

def process_multi_po(driver ,po_obj:MultiPOParse):
    po_tables = po_obj.extracted_data.get('po_tables')
    invoice_nos = po_tables.keys()
    print(invoice_nos)
    for invoice in invoice_nos:
        items_data = po_tables[invoice].get('po_numbers')
        print(items_data)
        for idx,item_data in enumerate(items_data):
            if idx == 0:
                continue
            else:
                data_to_send = po_obj.get_item_info(int(item_data))
                totals = po_tables[invoice].get('totals')
                csv_obj = po_tables[invoice].get('csv_obj')
                data_to_send['csv_obj'] = csv_obj
                data_to_send.update(totals)
                print(data_to_send)
                if not data_to_send.get('hs_code'):
                    continue
                else:
                    add_item(driver, 1,data=data_to_send)
                    
                    if idx == 0:
                        # Non_Duty_Paid_Info(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
                        print(f"Adding HS Code Wise Tables for hs code {data_to_send.get('hs_code')}")
                        Non_Duty_Paid_Info_multi_po(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
                        print(f"GD Completed For Item : {item_data}")
                    else:
                        Non_Duty_Paid_Info_multi_po(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
            
    return idx
process_multi_po(driver=driver,po_obj=po_parser)

In [2]:
from helpers import *

multi = MultiPOParse(path=fty_po_pdf_path,csv_path=csv_po_path,des_path=desc_po_path)
multi.extracted_data

Searching for CSV file for invoice number 508645 in c:\Users\DELL\Downloads\PWSAutomate\multi\multipos...
desc.xlsx
EFS508645.csv
Searching for CSV file for invoice number 520064 in c:\Users\DELL\Downloads\PWSAutomate\multi\multipos...
desc.xlsx
EFS508645.csv
EFS520064.csv


{'po_tables': {'508645': {'po_numbers': ['0134793136',
    '0135021120',
    '0135037910',
    '0135038956',
    '0135095361',
    '0135151371',
    '0135153369',
    '0135157520',
    '0135201839',
    '0135201858',
    '0135239107',
    '0135253471'],
   'totals': {'Quantity': 6847.0,
    'Carton': 386.0,
    'Gross Weight': 2470.0,
    'Net Weight': 2079.0,
    'Net Net Weight': 1970.0,
    'PO Net Amount': 41535.39,
    'VAT': 0.0,
    'invoice_number': '508645'},
   'csv_obj': <helpers.excelhelper.CSVDataExtractor at 0xd3bb6e8>},
  '520064': {'po_numbers': ['0134683620',
    '0135094105',
    '0135094154',
    '0135095335',
    '0135142747',
    '0135142758',
    '0135152000',
    '0135152808',
    '0135239121',
    '0135239245',
    '0135253219',
    '0135253457'],
   'totals': {'Quantity': 4739.0,
    'Carton': 381.0,
    'Gross Weight': 2000.0,
    'Net Weight': 1617.0,
    'Net Net Weight': 1541.0,
    'PO Net Amount': 34758.27,
    'VAT': 0.0,
    'invoice_number': '520064'},

In [3]:
print(multi.get_item_info())

[{134793136: {'PCS': 401, 'CTNS': 26, 'Desc': 'Womens 70% Recycle Polyester 26% Viscos Ryan Knit T-Shirts Dyed', 'PO_Amount': 3304.24}}, {135021120: {'PCS': 32, 'CTNS': 4, 'Desc': 'Mens 100% Cotton Knit Pants Dyed', 'PO_Amount': 276.48}}, {135037910: {'PCS': 660, 'CTNS': 57, 'Desc': 'Womens 50% Cotton 43% Recycled Polyester 7% Elastic Knit Pants Dyed', 'PO_Amount': 6223.8}}, {135038956: {'PCS': 585, 'CTNS': 61, 'Desc': 'Mens 82% Cotton 18% Recycled Polyester Knit Pullover Dyed.', 'PO_Amount': 8055.45}}, {135095361: {'PCS': 774, 'CTNS': 22, 'Desc': 'Men 100% Cotton Knit T-Shirt White', 'PO_Amount': 2863.8}}, {135151371: {'PCS': 1318, 'CTNS': 35, 'Desc': 'MEN 100% Cotton Knit T-Shirts Dyed', 'PO_Amount': 5074.3}}, {135153369: {'PCS': 422, 'CTNS': 44, 'Desc': 'Womens 100% Cotton Knit Jackets Dyed.', 'PO_Amount': 2890.7}}, {135157520: {'PCS': 100, 'CTNS': 10, 'Desc': 'Men 100% cotton knit T-shirt Dyed', 'PO_Amount': 696}}, {135201839: {'PCS': 199, 'CTNS': 21, 'Desc': 'Mens 70% Cotton 30% R

In [ ]:
driver = main(data)

In [ ]:
def process_duty_drawback(driver,description):
    table = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_pnldutydrawback"))
        )

    # Find all rows in the table body
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Iterate through the rows, skipping the header
    for row in rows[2:]:
        cells = row.find_elements(By.TAG_NAME, "td")
        if 'dye' in description.lower() and 'dyed or printed' in cells[1].text.lower():
            cells[0].click()
            return True
        elif 'bleach' in description.lower() and 'bleached blended' in cells[1].text.lower():
            cells[0].click()
            return True
        elif '100% polyester' in description.lower() and 'grey blended' in cells[1].text.lower():
            cells[2].click()
            return True
    else:
        row = rows[2]
        cells = row.find_elements(By.TAG_NAME, "td")
        cells[0].click()
        return True
process_duty_drawback(driver,'bleach Unisex Infants 78% Cotton 22% Recycled Polyester')


In [4]:
driver = main(data)

hi
For transaction_id 1 Cookies Done...
For transaction_id 1 Selecting GDS...
Found traders_button.
iframe Switched.
Found company_element.
Switched to default_content.
Found goods_declaration.
Found Create_Export_GD.
Found ctl00_ContentPlaceHolder2_GdSelectionExport1_ddlConsignmentType dropdown_elem.
Found ctl00_ContentPlaceHolder2_GdSelectionExport1_ddlDeclarationType dropdown_elem.
Found Create_button.
Clicked Create_button.
GD_status: True, GD_error: 
For transaction_id 1 Filling the Form...
Found ctl00_ContentPlaceHolder2_BasicInfoUc1_ddlCollectorate dropdown_elem.
Found ctl00_ContentPlaceHolder2_BasicInfoUc1_ddlModeOfTransport dropdown_elem.
Found ctl00_ContentPlaceHolder2_ConsigneeInfoUc1_txtConsignorName element.
Found ctl00_ContentPlaceHolder2_ConsigneeInfoUc1_txtConsignorAddress element.
Found ctl00_ContentPlaceHolder2_GdInfoSeaUc_ddlConsignmentMode dropdown_elem.
Found ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtBlNo element.
Found ctl00_ContentPlaceHolder2_GdInfoSeaUc_txtBlDat

In [8]:
def process_duty_drawback(driver,description):
    table = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder2_ItemsInfoDetailUc1_pnldutydrawback"))
        )

    # Find all rows in the table body
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Iterate through the rows, skipping the header
    for row in rows[2:]:
        cells = row.find_elements(By.TAG_NAME, "td")
        if 'dye' in description.lower() and 'dyed or printed' in cells[1].text.lower():
            cells[0].click()
            return True
        elif 'bleach' in description.lower() and 'bleached blended' in cells[1].text.lower():
            cells[0].click()
            return True
        elif '100% polyester' in description.lower() and 'grey blended' in cells[1].text.lower():
            cells[2].click()
            return True
    else:
        row = rows[2]
        cells = row.find_elements(By.TAG_NAME, "td")
        cells[0].click()
        return True
process_duty_drawback(driver,'bleach Unisex Infants 78% Cotton 22% Recycled Polyester')


True

In [2]:
def find_hs_code(description):
    description = description.lower().strip().replace('\n', '')
    print(description)
    material_category = determine_category(description)

    garment_type = None
    gender_category = None

    if 't-shirt' in description or 't shirt' in description:
        garment_type = 'T-Shirts'
    elif 'shirt' in description:
        garment_type = 'Shirts'
    elif 'pant' in description or 'shorts' in description:
        garment_type = 'Pants'
    elif 'jacket' in description:
        garment_type = 'Jackets'
    elif 'track suit' in description:
        garment_type = 'Track Suits'
    elif 'pullover' in description:
        garment_type = 'Pullovers'

    if 'unisex' in description:
        gender_category = 'Women'
    elif 'women' in description or 'ladies' in description or 'girl' in description:
        gender_category = 'Women'
    elif 'men' in description or 'gentle' in description or 'boy' in description or 'kid' in description:
        gender_category = 'Men'
    
    print(f"{gender_category} {material_category} {garment_type}")
    if material_category and garment_type and gender_category:
        key = f"{gender_category} {material_category} {garment_type}"
        return hs_codes.get(key)

    return None

def process_multi_po(driver ,po_obj:MultiPOParse):
    po_tables = po_obj.extracted_data.get('po_tables')
    invoice_nos = po_tables.keys()
    print(invoice_nos)
    for invoice in invoice_nos:
        items_data = po_tables[invoice].get('po_numbers')
        print(items_data)
        for idx,item_data in enumerate(items_data):
            if idx == 0:
                continue
            else:
                data_to_send = po_obj.get_item_info(int(item_data))
                totals = po_tables[invoice].get('totals')
                csv_obj = po_tables[invoice].get('csv_obj')
                data_to_send['csv_obj'] = csv_obj
                data_to_send.update(totals)
                print(data_to_send)
                if not data_to_send.get('hs_code'):
                    continue
                else:
                    add_item(driver, 1,data=data_to_send)
                    
                    if idx == 0:
                        # Non_Duty_Paid_Info(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
                        print(f"Adding HS Code Wise Tables for hs code {data_to_send.get('hs_code')}")
                        Non_Duty_Paid_Info_multi_po(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
                        print(f"GD Completed For Item : {item_data}")
                    else:
                        Non_Duty_Paid_Info_multi_po(driver,data_to_send.get('csv_obj'),hs_code=data_to_send.get('hs_code'),elem_index=idx+1)
            
    return idx
process_multi_po(driver=driver,po_obj=po_parser)

dict_keys(['508645', '520064'])
['0134793136', '0135021120', '0135037910', '0135038956', '0135095361', '0135151371', '0135153369', '0135157520', '0135201839', '0135201858', '0135239107', '0135253471']
mens 100% cotton knit pants dyed
Men 100% Pants
{'CTNS': 4, 'description': 'Mens 100% Cotton Knit Pants Dyed', 'hs_code': '6103.4200', 'csv_obj': <helpers.excelhelper.CSVDataExtractor object at 0x0B453088>, 'Quantity': 6847.0, 'Carton': 386.0, 'Gross Weight': 2470.0, 'Net Weight': 2079.0, 'Net Net Weight': 1970.0, 'PO Net Amount': 41535.39, 'VAT': 0.0, 'invoice_number': '508645'}
For transaction_id 1 Adding Item...
Exception: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.89)
Stacktrace:
	GetHandleVerifier [0x00007FF74E909632+30946]
	(No symbol) [0x00007FF74E8BE3C9]
	(No symbol) [0x00007FF74E7B6FDA]
	(No symbol) [0x00007FF74E78CB85]
	(No symbol) [0x00007FF74E8337A7]
	(No symbol) [0x00007FF74E84A771]
	(No symb

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.89)
Stacktrace:
	GetHandleVerifier [0x00007FF74E909632+30946]
	(No symbol) [0x00007FF74E8BE3C9]
	(No symbol) [0x00007FF74E7B6FDA]
	(No symbol) [0x00007FF74E78CB85]
	(No symbol) [0x00007FF74E8337A7]
	(No symbol) [0x00007FF74E84A771]
	(No symbol) [0x00007FF74E82C813]
	(No symbol) [0x00007FF74E7FA6E5]
	(No symbol) [0x00007FF74E7FB021]
	GetHandleVerifier [0x00007FF74EA3F83D+1301229]
	GetHandleVerifier [0x00007FF74EA4BDB7+1351783]
	GetHandleVerifier [0x00007FF74EA42A03+1313971]
	GetHandleVerifier [0x00007FF74E93DD06+245686]
	(No symbol) [0x00007FF74E8C758F]
	(No symbol) [0x00007FF74E8C3804]
	(No symbol) [0x00007FF74E8C3992]
	(No symbol) [0x00007FF74E8BA3EF]
	BaseThreadInitThunk [0x00007FFB17847374+20]
	RtlUserThreadStart [0x00007FFB17B1CC91+33]
